# **Introduction to In-Context Learning Demo: Translation of Fictional Movie Languages**

This notebook provides a step-by-step walkthrough of an interactive demo that examines how Large Language Models (LLMs) **can learn to translate directly from a small dataset of bilingual examples**. Instead of focusing on numerical or mathematical patterns, this experiment explores how an LLM can absorb the linguistic structure from fictional cinematic languages such as Na’vi, Klingon, Dothraki, and Elvish and use those examples to translate new phrases it has never encountered before. The goal is to show how an LLM without being explicitly told the grammar, morphology, or vocabulary can observe several short English↔fictional language pairs and infer enough structure to translate new test phrases accurately.

**Because modern LLMs are already trained on many real languages, we use fictional ones to clearly demonstrate that the model is relying on in-context learning, not prior knowledge.**

## **Overview**

This demonstration guides you through:

- Loading a dataset of English and fictional-language phrase pairs

- Preparing example translations for in-context learning

- Constructing prompts containing these demonstration pairs

- Querying the Gemini 2.0/2.5 Flash model to translate test phrases from the dataset

- Allowing users to try custom sentences and evaluate the model’s inferred translation ability

The result is a tabular visualization of how LLMs perform translation through pattern inference—without prior knowledge of the invented languages.

## **Background**

This demo focuses on translation tasks using a curated dataset containing short English sentences paired with their equivalents from several well-known fictional movie and TV languages:

- Na’vi (Avatar)

- Klingon (Star Trek)

- Dothraki (Game of Thrones)

- Elvish (Lord of The Rings)

For each language, we provide a small set of example phrase pairs. These examples are given to the LLM as demonstrations of the “kind” of mapping we expect it to infer and apply. Separate test phrases—never shown in the examples—are used to measure how well the model generalizes.

What makes this setup interesting is that the model receives only a dataset of English ↔ fictional-language sentence pairs and has no prior-knowledge of syntax, grammar, or language structure. Also, each fictional language has a unique structure, morphology, and word order.
The test phrases include vocabulary combinations not directly seen in the example pairs. Hence, the model must generalize beyond memorization to produce reasonable translations

This setting provides an intuitive benchmark of whether an LLM can identify and extrapolate latent linguistic patterns purely from exposure to a small translation dataset—showcasing the power of in-context learning for rapid, example-driven language adaptation.

## **Let's Take a Look at an Example Train Phrase from the Dataset**

The example below shows a sample training phrase from the dataset, containing an English sentence paired with its Na’vi translation. The model sees many pairs like this or ones that look like ```"English":"Fictional Language"``` in the prompt. These examples demonstrate the style, word order, and phrasing patterns of the fictional language. The test phrases that follow challenge the LLM to apply the same inferred patterns to new sentences, producing reasonable translations without any grammar rules, dictionaries, or prior knowledge of Na’vi.

<img src="https://raw.githubusercontent.com/intro-to-icl/intro-to-icl.github.io/refs/heads/master/static/images/translations.png" height="500">


## **LLM as the Translator**

In this demo, we use Gemini 2.0/2.5 Flash in non-reasoning mode. This ensures the model focuses on pattern matching within the provided examples rather than relying on explicit linguistic reasoning or prior knowledge of real-world languages.

The workflow is simple:
- Provide several English ↔ fictional-language phrase pairs
- Append new test sentences that are not included in the training examples
- Ask the model to translate these unseen phrases into the target fictional language (or back to English)
- The LLM behaves like a pattern-driven translation engine—absorbing vocabulary, structure, and phrasing style from the examples and applying them to new sentences.

This approach highlights how LLMs can infer translation behavior purely through in-context exposure, without needing any predefined grammar rules or dictionaries.

## **Code Overview**
The implementation is organized into a modular structure, with each component responsible for a different stage of the translation pipeline. This design separates data loading, visualization, prompt construction, LLM inference, and result interpretation, making the system easy to understand, modify, and extend.

## **Before Running the Demo, Follow the Instructions Below**
To run the full experiment:
1. Ensure all dependencies are imported and installed.
2.	Visit Google AI Studio (https://aistudio.google.com/) to obtain your Gemini API key.
3.	Once API key is generated, copy and paste it into the demo when prompted.

#### **If having trouble creating an API Key, follow the link below for instructions:**
* #### **[Instructions on How to Obtain a Gemini API Key](https://docs.google.com/document/d/17pgikIpvZCBgcRh4NNrcee-zp3Id0pU0vq2SG52KIGE/edit?usp=sharing)**



In [ ]:
#@title **Import Necessary Libraries**
import numpy as np
import random
import matplotlib.pyplot as plt
from google import genai
from google.genai import types
import time
import re
import itertools
import math
import ast
import json
import ipywidgets as widgets
from IPython.display import display
import os
import getpass
import torch
from transformers import BertTokenizer, BertModel
import numpy as np
from tqdm import tqdm
import pandas as pd


In [ ]:
#@title **Setting Up Gemini Client**
apikey = getpass.getpass("Enter your Gemini API Key: ")
client = genai.Client(api_key=apikey)

Enter your Gemini API Key: ··········


In [ ]:
#@title **Choose a Model**
model_dropdown = widgets.Dropdown(
    options=[
        ("Gemini 2.5 Flash", "gemini-2.5-flash"),
        ("Gemini 2.0 Flash", "gemini-2.0-flash")
    ],
    description="Model:",
    value="gemini-2.5-flash",
    style={'description_width': 'initial'}
)

confirm_button = widgets.Button(
    description="Confirm Selection"
)

output = widgets.Output()

model_name = None
batch_size = None

def on_confirm_click(b):
    global model_name, batch_size

    model_name = model_dropdown.value

    if model_name == "gemini-2.5-flash":
        batch_size = 15
    else:
        batch_size = 10

    with output:
        output.clear_output()
        print(f"\nSelected model: {model_name}")

confirm_button.on_click(on_confirm_click)

display(model_dropdown, confirm_button, output)

Dropdown(description='Model:', options=(('Gemini 2.5 Flash', 'gemini-2.5-flash'), ('Gemini 2.0 Flash', 'gemini…

Button(description='Confirm Selection', style=ButtonStyle())

Output()

## **Fictional Language Translation Demo Structure**

### **Setup Instructions**

Run all cells in order to initialize the environment and load the required dependencies. Once setup is complete, execute the Demo cell. You will then be prompted to select a fictional language from the dropdown menu and click Confirm to begin the translation process.

### **Running the Demo**

After you confirm your selection, the model will infer the linguistic structures and stylistic patterns from the provided training examples, then translate a test English phrase into the selected fictional language. It will also display a table that includes the original English phrases, the expected translations, and the model-generated translations.*

### **Evaluate Translation Quality**

Once the translation output appears, run the **Translation Evaluation** cell.  This step will automatically compare the model’s translations with the expected results and calculate their accuracy.

### **Bonus: Custom Phrase Translation**

Use the **Final Cell** to input your own English phrase.  
Then, choose any fictional language from the dropdown to generate a custom translation instantly!

## **Set Up Language Database for the Fictional Movie Languages**

These four cells serve as the data loaders for each fictional movie language used in the demo. Each cell defines a curated dataset containing example English ↔ fictional-language phrase pairs, along with a separate set of held-out test sentences.

For every language—Na’vi, Klingon, Dothraki, and Elvish—we provide: a training list of aligned phrase pairs, corresponding test list containing phrases not shown in the training examples, and use a consistent format that allows the LLM to observe patterns and then generalize to new inputs

These datasets form the foundation for the in-context learning process, demonstrating how the model infers translation behavior based solely on the examples provided.

In [ ]:
#@title **Na'vi Train-Test Data**
navi_phrases = {
    "oops": "'ä'",
    "ow": "'ak",
    "ouch": "'ak",
    "hello": "kaltxì",
    "hi": "kxì",
    "goodbye": "kiyevame",
    "thank you": "irayo",
    "thanks": "irayo",
    "yes": "srane",
    "no": "kehe",
    "please": "rutxe",
    "I see you": "oel ngati kameie",
    "welcome": "zola'u nìprrte'",
    "congratulations": "seykxel sì nìtram",
    "bummer": "yewla",
    "excuse me": "hitxoa",
    "wait": "pey",
    "stop": "ftang",
    "ready": "alaksi",
    "I don't know": "kemum",
    "sorry": "ngaytxoa",
    "I": "oe",
    "me": "oe",
    "you": "nga",
    "he": "poan",
    "she": "poe",
    "it": "tsaw",
    "we (exclusive)": "ayoe",
    "we (inclusive)": "ayoeng",
    "they": "ayfo",
    "everyone": "frapo",
    "everything": "fra'u",
    "something": "uo",
    "nothing": "ke'u",
    "one": "'aw",
    "two": "mune",
    "three": "pxey",
    "four": "tsing",
    "five": "mrr",
    "six": "pukap",
    "seven": "kinä",
    "eight": "vol",
    "nine": "nayn",
    "zero": "kew",
    "first": "'awve",
    "second": "muve",
    "once": "'awlo",
    "twice": "melo",
    "person": "tute",
    "people": "Na'vi",
    "man": "tutan",
    "woman": "tuté",
    "boy": "'evengan",
    "girl": "'evenge",
    "child": "'eveng",
    "kid": "'evi",
    "mother": "sa'nok",
    "mommy": "sa'nu",
    "father": "sempul",
    "daddy": "sempu",
    "son": "'itan",
    "daughter": "'ite",
    "friend": "'eylan",
    "enemy": "kxutu",
    "leader": "eyktan",
    "warrior": "tsamsiyu",
    "hunter": "taronyu",
    "teacher": "karyu",
    "student": "numeyu",
    "family": "soaia",
    "clan": "olo'",
    "husband": "muntxatan",
    "wife": "muntxate",
    "ancestor": "pizayu",
    "body": "tokx",
    "head": "re'o",
    "face": "key",
    "eye": "nari",
    "ear": "mikyun",
    "nose": "ontu",
    "mouth": "kxa",
    "tooth": "sre'",
    "tongue": "ftxì",
    "neck": "pewn",
    "shoulder": "'etnaw",
    "arm": "pxun",
    "hand": "tsyokx",
    "finger": "zekwä",
    "chest": "heyr",
    "heart": "txe'lan",
    "stomach": "ngäng",
    "leg": "kinam",
    "foot": "venu",
    "tail": "kxetse",
    "skin": "ta'leng",
    "blood": "reypay",
    "bone": "txärem",
    "brain": "eltu",
    "sky": "taw",
    "sun": "tsawke",
    "moon": "oare",
    "star": "tanhì",
    "cloud": "piwopx",
    "rain": "tompa",
    "wind": "hufwe",
    "snow": "herwì",
    "fire": "txep",
    "water": "pay",
    "land": "atxkxe",
    "ground": "kllte",
    "forest": "na'rìng",
    "tree": "utral",
    "leaf": "rìk",
    "flower": "syulang",
    "grass": "zeswa",
    "mountain": "ram",
    "river": "kilvan",
    "ocean": "txampay",
    "animal": "ioang",
    "plant": "'ewll",
    "rock": "tskxe",
    "banshee": "ikran",
    "direhorse": "pa'li",
    "time": "krr",
    "now": "set",
    "today": "fìtrr",
    "tomorrow": "trray",
    "yesterday": "trram",
    "day": "trr",
    "night": "txon",
    "morning": "rewon",
    "noon": "kxamtrr",
    "afternoon": "ha'ngir",
    "week": "kintrr",
    "month": "vospxì",
    "year": "zìsìt",
    "future": "zusawkrr",
    "past": "ftawnemkrr",
    "always": "frakrr",
    "never": "kawkrr",
    "soon": "ye'rìn",
    "color": "'opin",
    "blue": "ean",
    "green": "ean",
    "red": "tun",
    "orange": "tun",
    "yellow": "rim",
    "white": "teyr",
    "black": "layon",
    "grey": "ngul",
    "purple": "'om",
    "dark": "vawm",
    "be": "lu",
    "do": "kem si",
    "go": "kä",
    "come": "za'u",
    "run": "tul",
    "walk": "tiran",
    "fly": "tswayon",
    "swim": "slele",
    "jump": "spä",
    "sit": "heyn",
    "stand": "kllkxem",
    "sleep": "hahaw",
    "wake up": "tìtxen si",
    "eat": "yom",
    "drink": "näk",
    "speak": "plltxe",
    "listen": "yune",
    "see": "tse'a",
    "look": "nìn",
    "hear": "stawm",
    "smell": "syam",
    "touch": "'ampi",
    "feel": "'efu",
    "think": "fpil",
    "know": "omum",
    "understand": "tslam",
    "learn": "nume",
    "teach": "kar",
    "love": "txasunu",
    "hate": "ve'kì",
    "want": "new",
    "need": "kin",
    "take": "munge",
    "give": "tìng",
    "make": "ngop",
    "break": "kxakx",
    "open": "piak si",
    "close": "tstu si",
    "hunt": "taron",
    "kill": "tspang",
    "die": "terkup",
    "fight": "wem",
    "help": "srung si",
    "play": "uvan si",
    "sing": "rol",
    "smile": "lrrtok si",
    "cry": "tsngawvik",
    "laugh": "hangham",
    "good": "siltsan",
    "bad": "fe'",
    "big": "tsawl",
    "small": "hì'i",
    "long": "ngim",
    "short": "pup",
    "hot": "som",
    "cold": "wew",
    "fast": "win",
    "slow": "kì'ong",
    "heavy": "ku'up",
    "light (weight)": "syo",
    "hard": "txa'",
    "soft": "hewne",
    "new": "mip",
    "old": "koak",
    "beautiful": "lor",
    "ugly": "narvä'",
    "happy": "nitram",
    "sad": "keftxo",
    "angry": "sti",
    "afraid": "txopu",
    "brave": "tstew",
    "strong": "txur",
    "weak": "meyp",
    "easy": "ftue",
    "difficult": "ngäzik",
    "correct": "eyawr",
    "wrong": "keyawr",
    "true": "ngay",
    "false": "tsleng",
    "alive": "rusey",
    "dead": "kerusey",
    "full": "teya",
    "empty": "mek",
    "hungry": "ohakx",
    "thirsty": "väng",
    "home": "kelku",
    "house": "kelku",
    "path": "fya'o",
    "bridge": "semkä",
    "village": "tsray",
    "school": "numtseng",
    "food": "syuve",
    "meat": "vey",
    "fruit": "mauti",
    "water": "pay",
    "bow": "tsko",
    "arrow": "swizaw",
    "knife": "tstal",
    "spear": "tukru",
    "clothing": "pxen",
    "shoe": "hawnven",
    "book": "puk",
    "message": "upxare",
    "name": "tstxo",
    "word": "lì'u",
    "language": "lì'fya",
    "song": "tìrol",
    "picture": "rel",
    "and": "sì",
    "but": "slä",
    "or": "fu",
    "if": "txo",
    "because": "talun",
    "with": "hu",
    "without": "luke",
    "for": "fpi",
    "from": "ftu",
    "to": "ne",
    "in": "mì",
    "on": "sìn",
    "under": "ìo",
    "above": "io",
    "before": "eo",
    "after": "maw",
    "near": "sim",
    "far": "lìm",
    "may the Great Mother smile upon our first meeting.": "Awvea ultxari ohengeyä, Nawma Sa'nok lrrtok siveiyi.",
    "Not bad. You still sound a bit too formal, though.": "Tsun tivam. Aylì'u ngian nì'it skepek lu.",
    "I studied for five years but there is much to learn.": "Zìsìt amrr ftolia ohe, slä zene fko nivume nìtxan.",
    "These Demons!": "Fayvrrtep!",
    "These Sky People!": "Faysawtute!",
    "Forgive me. My brother, my brother.": "Oeru txoa livu. Ma oeyä tsmukan, ma oeyä tsmukan.",
    "Forgive me, my brother. May your spirit run with the Great Mother. Forgive me.": "Oeru txoa livu, ma oeyä tsmukan. Hu nawma sa'nok tivul ngeyä tirea. Oeru txoa livu.",
    "No, go back!": "Kehe, go back!",
    "Stop that!": "Ftang nga!",
    "Please. No!": "Rutxe. No!",
    "Seeds of the great tree!": "Atokirina'! Atokirina'!",
    "(That) has been a sign.": "Lolu aungia.",
    "Calm, people, calm.": "Mawey, Na'viya! Mawey!",
    "What are you doing, Tsu'tey?": "Ma Tsu'tey! Kempe si nga?",
    "These demons are forbidden here!": "Fayvrrtep fìtsenge lu kxanì!",
    "There has been a sign! This is a matter for the Tsahìk!": "Aungia lolu! Tsahìkur txele lu!",
    "Bring him!": "Pot zamunge!",
    "Let's ride!": "Makto ko!",
    "What is he?": "'Upe lu po?",
    "Release him!": "Pot lonu!",
    "Why four?": "Tsìng pelun?",
    "Father, I see you.": "Ma sempul, oel ngati kameie.",
    "This creature, why did you bring him here?": "Fìswiräti, ngal pelun molunge fìtseng?",
    "I was going to kill him, but there was a sign from Eywa.": "Oel pot tspìmìyang, tsakrr za'u aungia ta Eywa.",
    "I have said: “No dreamwalker will come here.”": "Poltxe oe, san zene kea uniltìranyu, ke ziva'u fìtseng.",
    "His alien smell fills my nose.": "((ketepeu)) fahew akewong.",
    "No!": "Kehe!",
    "Release this creature! Step back!": "Tsaswiräti lonu! Aynga neto rivikx!",
    "I will look at this alien.": "Fìketuwongti oel stìyeftxaw.",
    "A warrior? I could kill him easily!": "Tsamsiyu pak? Pot tsun oe tspivang nìftue.",
    "This is the first warrior dreamwalker we have seen.": "Fìpo lu 'awvea, uniltìranyu-tsamsiyu, a tsole'a awngal.",
    "We need to learn more about him.": "Pori awngaru lu tìkin, a nume nì'ul.",
    "My daughter, you will teach him our way...": "Ma 'ite, awngeyä fya'ori zene nga sänume sivi poru...",
    "...to speak and walk as we do.": "...fte tsivun pivlltxe sì tivìran na ayoeng.",
    "Why me? That's not fair.": "Oeru pelun? Ke lu muiä.",
    "It is decided.": "Fkol pole'un fì'ut",
    "Dammit!": "Wiya!",
    "Eat (your) meal.": "Yom wutsot.",
    "Neytiri of the Tskaha, daughter of Mo'at.": "Neytiri te Tskaha Mo'at'ite.",
    "These aliens try to look like people...": "Fayhetuwong fmi Na'vina livam...",
    "But they can't.": "Slä ke tsun.",
    "He seems dim to me.": "(?) oeru po snumìna latsam.",
    "And his eyes are too small.": "Peyä menarisì nìhawng lu hì'i",
    "Good.": "Sìltsan.",
    "Good, Pale, Good.": "Sìltsan, Pale, sìltsan.",
    "This alien will learn nothing.": "Fìketuwongìl ke nayume ke'ut.",
    "A rock sees more. Look at him.": "Nì'ul kame tskxe. Poru tìng nari.",
    "Go.": "Kä.",
    "Okay, okay, Seze.": "Tam tam, Seze.",
    "Nose, ear, eye. Eyeee!": "Ontu, mikyun, nari. Naaariii!",
    "Stronger.": "Txur nì'ul.",
    "Moron.": "Skxawng.",
    "I see you.": "Oel ngati kameie.",
    "Hello, children.": "Kaltxì, ma evi.",
    "Look how big you are.": "Tsawl nìtxan nang.",
    "You are so pretty.": "Sevin nìtxan lu aynga nang.",
    "A memory from English.": "('Ok-'it) ta 'Ìnglìsì.",
    "I See you, Brother, and thank you.": "Oel ngati kameie, ma tsmukan, ulte ngaru seiyi irayo.",
    "Your spirit goes with Eywa, your body stays behind to become part of the People.": "Ngari hu Eywa salew tirea, tokx 'ì'awn slu Na'viyä hapxì.",
    "Come!": "Za'u!",
    "With me.": "Oehu.",
    "Keep up!": "Latsi!",
    "Let's go!": "Kivä ko!",
    "(I) See you (two).": "Mengati kame.",
    "Warrior, don't be scared, warrior!": "Tsamsiyu! Txopu rä'ä si, tsamsiyu!",
    "Don't do that.": "Tsakem rä'ä si.",
    "That moron's going to die.": "Tìyerkup skxawng.",
    "Don't be afraid, warrior.": "Txopu rä'ä si, tsamsiyu.",
    "Go, go!": "Kä, kä!",
    "Bond Jake! Bond!": "Tsaheylu Jake! Tsaheylu!",
    "Yes!": "Srane!",
    "(Angry snarl)": "Oìsss!",
    "Pay attention!": "Eltu si!",
    "Good, Seze.": "Sìltsan, Seze.",
    "I here. Toruk here. I turn, and toruk, he almost grabs me.": "Oe fìtseng. Toruk fìtseng. Oe mìn, toruk, stum niä pol oeti.",
    "This seed is growing upon us. Her path is with Tsu’tey!": "Sìn ayoeng fìrina' apxa slivu. Peyä fya’o Tsu’teyhu lu!",
    "O wise worm, who eats the Sacred Tree.": "Ma txantslusama ngawng, a swoka utralti yom.",
    "Grant this worthy warrior true vision.": "Fìtaronyur apxan teswotivìng äiet angay.",
    "Grant true vision. Grant true vision. In the plants, grant true vision to him.": "Teswotivìng äiet angay. Teswotivìng äiet angay. Mì ayewll, poru teswotìng äiet angay.",
    "You are now a son of the Omaticaya. You are part of The People.": "Ngenga 'itan Omatikayaä luyu set. Na'viyä luyu hapxì.",
    "Jake!": "Ma Jake!",
    "Tsu'tey will lead the war party.": "Tsampongut Tsu'teyìl iveyk.",
    "Stop! Please! This will only make it worse.": "Ftang! Rutxe! Fìkem nì'ul kawng sìyi nì'aw.",
    "We will strike them in the heart!": "Foti awngal tìyakuk, mì te'lan!",
    "No! Tsu'tey! Don't!": "Kehe! Tsu'tey! Rä'ä si!",
    "I have something to say. To all of you.": "Lu aylì'u ngaru. Ayngaru nìwotx.",
    "You see? It is a demon in a false body!": "Tse'a srak? Fìpo lu vrrtep a mì sokx atsleng!",
    "He must not live!": "Zenke rivey!",
    "We are mated before Eywa. It is done.": "Eo Eywa moe muntxa slolu. Lu hasey.",
    "Neytiri was promised to me.": "Neytiriti fkol pänutolìng oer.",
    "Everything is changing.": "Fra’u leratängem.",
    "Everything is being destroyed.": "Fra’ut fkol skera’a.",
    "These aliens kill everything they touch.": "Fayhetuwongìri, tsat ‘ampi, tspìyang.",
    "Like poison.": "Pxel txum.",
    "Stop! Stop! This challenge is not right.": "Ftang! Ftang! Fìfpeio ke lu muiä.",
    "Urgh! Hiss! Hiss!": "Sau! Oìsss! Oìsss!",
    "This is a demon that’s in a false body!": "Fìpo lu vrrtep a lu mì sokx atsleng!",
    "It is not right that it lives.": "Ke lu muiä fwa tsun rivey.",
    "War party!": "Tsampongu!",
    "Eytukan, I have something to say, to everyone.": "Ma Eytukan, lu oeru aylì'u frapor.",
    "Listen!": "Tìng mikyun!",
    "Speak, Jakesully.": "Pivlltxe, ma Jakesully.",
    "A great evil is upon us.": "Eo ayoeng lu txana tìkawng.",
    "The Sky People are coming...": "Sawtute zera'u...",
    "...to destroy Hometree. Destroy!": "..fte fol Kelutralti, skiva'a! Skiva'a!",
    "(Exclamation of consternation.)": "Au!",
    "They will be here soon.": "Pìyate fìtseng ye'rìn.",
    "You will never be one of the people!": "Kawkrr! Ke slayu nga Na'viyä hapxì!",
    "Never of The People, never!": "Kawkrr Na'viyä, kawkrr!",
    "Daughter.": "Ma 'ite.",
    "Bind them.": "Mefoti yìm.",
    "Leave now!": "Set hivum!",
    "Eytukan, they're attacking you.": "Ma Eytukan, ngati 'eko.",
    "Destroy!": "Skiva'a!",
    "You must leave!": "Ayngari zene hivum!",
    "Have no fear! Have no fear!": "Txopu rä'ä si! Txopu rä'ä si!",
    "Ride the Ikran. Attack from above.": "Ikranti makto. 'eko ta'em.",
    "Attack!": "'Eko!",
    "Everyone to the forest!": "Frapo ne wrrpa!",
    "Don't wait now!": "Rä'ä set pey!",
    "Father. Father, I need you.": "Ma sempul! Ma sempul, ngati oel kin.",
    "Daughter, take my bow.": "Ma 'ite, tskoti munge.",
    "Protect the people.": "Omatikayaru tìhawnu sivi.",
    "Tsahìk, stop.": "Ma tsahìk, ftang nga.",
    "Rider of last shadow?": "Toruk Makto?",
    "Tsu'tey of the Rongloa, son of Ateyo.": "Ma Tsu'tey te Rongloa, Ateyo'itan.",
    "I stand before you, ready to serve the Omatikaya people.": "Eo ngenga kllkxem ohe, alaksì Omatikayaru, kìte'e sivi.",
    "Hear us please, Great Mother.": "Tìng mikyun ayoheru rutxe, ma Nawma Sa'nok.",
    "Eywa, help her.": "Srung si poeru, ma Eywa.",
    "Take this spirit into you.": "Pori tireati, munge mì nga,",
    "And breathe her back to us.": "Ulte tìng ayoer nì'eyng hu ngeyä ya.",
    "Let her walk among us,": "Tivìran po ayoekip,",
    "As one of The People!": "Na Na'viyä hapxì!",
    "I lose myself before Eywa.": "Eo Eywa oe 'ia.",
    "It is finished!": "Lu hasey!",
    "The Sky People have sent us a message...": "Fpole' sawtutel 'upxaret.",
    "...that they can take whatever they want...": "Ayoeri tsat new.",
    "...and no one can stop them.": "Tsun mivunge.",
    "Well, we will send them a message.": "Slä awngal 'upxaret fpìye' for.",
    "You ride out as fast as the wind can carry you.": "Kämakto nìwin, ayngati spivule hufwel.",
    "You tell the other clans to come.": "Ayolo'ru alahe peng ziva'u.",
    "Tell them Toruk Makto calls to them.": "For(u) peng syeraw toruk makto.",
    "You fly now, with me!": "Tswayon set oehu!",
    "My brothers, my sisters!": "Ma smukan, ma smuke!",
    "And we will show the Sky People...": "Sawtute wìyintxu ayoeng.",
    "...that they cannot take whatever they want!": "Ke tsun fo fìkem sivi!",
    "And that this is our land!": "Fìtsenge... (lu) awngeyä!",
    "With that!": "Hu tsat!",
    "Let's ride!": "Makto ko!",
    "Attack! Attack! Attack!": "'Eko! 'Eko! 'Eko!",
    "I see you, Jakesully.": "Oel ngati kameie, Jakesully.",
    "Forgive me, my brother. Go now to the mother spirit...": "Oeru txoa livu, ma oeyä tsmukan. Kä set ne sa'nok tirea. Ngari hu Eywa salew tirea, tokx 'ì'awn slu Na'viyä hapxì.",
    "Mother.": "Ma sa'nok.",
    "It is finished.": "Lu hasey."
}

navi_eval = {
    "Just a tiny moment": "'awa swawtsyip",
    "Be hungry": "'efu ohakx",
    "What's been keeping you busy lately?": "'ìn nga fyape nìfkrr",
    "Slow is fine": "'ivong nìk'ong",
    "It's a foregone conclusion": "ätxäle pa(lu)lukanur",
    "How are things in your part of the world?": "aytele a ngeyä hapximì kifkeyä lu fyape?",
    "Be boring": "eltut heykahaw",
    "Good luck": "etripa syayvi",
    "Goodbye (Eywa be with you)": "Eywa ngahu",
    "This is Istaw": "fìpor syaw fko Istaw",
    "Quiet, young one": "fnu, ma 'evi",
    "Studying Na'vi is a ton of fun for me": "ftia oel lì'fyati leNa'vi nì'o' nìwotx",
    "Whether... or": "ftxey... fuke",
    "Next time from me": "hayalo ta oe",
    "Thanks": "irayo",
    "Generally": "ka wotx",
    "Hello (casual)": "kaltxì",
    "I'm not worthy": "ke pxan",
    "It's irrelevant": "ke tare",
    "I didn't get that": "ke tslolam",
    "Take your time": "ke zene win säpivi",
    "No need": "kea tìkin",
    "Right? (Isn't that so?)": "kefya srak",
    "See you again soon": "kiyevame",
    "At times": "krro krro",
    "Everybody": "ma frapo",
    "My son (address)": "maitan",
    "My daughter (address)": "maite",
    "Take care": "makto zong",
    "Get down": "ne kllte",
    "Tell me a bit more about yourself": "nga läpivawk nì'it nì'ul ko",
    "Who do you normally spend time with?": "nga pesuhu käteng nìtrrtrr",
    "I love you": "nga yawne lu oer",
    "Where did you come from?": "nga zola'u ftu peseng",
    "How old are you?": "ngari solalew polpxaya zìsìt",
    "Don't worry": "ngari txe'lan mawey livu",
    "How are you?": "ngaru lu fpom srak?",
    "What is your primary role in society?": "ngaru lu pefnetxintin nìtrrtrr",
    "To you the mastery": "ngaru tsulfä",
    "Who are you?": "ngenga lu tupe"
}

In [ ]:
#@title **Klingon Train-Test Data**
klingon_phrases = {
    "shoot": "bach",
    "shot": "bach",
    "fire (torpedo, rocket, missile)": "baH",
    "gunner": "baHwI'",
    "love, one who is loved": "bang",
    "metal": "baS",
    "honor": "batlh",
    "honored, with honor": "batlh",
    "orbit": "bav",
    "sit": "ba'",
    "suffer": "bech",
    "deflectors": "begh",
    "rifle": "beH",
    "watch": "bej",
    "be pleased": "bel",
    "pleasure": "bel",
    "be displeased": "belHa'",
    "years ago": "ben",
    "agony": "bep",
    "complain, object, gripe": "bep",
    "crew, crewman": "beq",
    "be irritable": "bergh",
    "female, woman": "be'",
    "girl": "be'Hom",
    "wife": "be'nal",
    "sister": "be'nI'",
    "half": "bID",
    "prison, jail": "bIghHa'",
    "they, them (incapable of language)": "bIH",
    "area below, area under": "bIng",
    "hundred thousand": "bIp",
    "water": "bIQ",
    "river": "bIQtIq",
    "ocean": "bIQ'a'",
    "be cold": "bIr",
    "be nervous, uneasy": "bIt",
    "break (rules)": "bIv",
    "module": "bobcho'",
    "shine, be shiny": "boch",
    "be born": "bogh",
    "be impatient": "boH",
    "nag": "boj",
    "accidentally, by accident": "bong",
    "alliance": "boq",
    "aide": "boQ",
    "assist": "boQ",
    "aide-de-camp": "boQDu'",
    "revenge": "bortaS",
    "collect": "boS",
    "prevent, block, prohibit": "bot",
    "center, middle": "botlh",
    "era": "bov",
    "court": "bo'DIj",
    "be lazy": "buD",
    "quit": "bup",
    "threaten": "buQ",
    "stomach": "burgh",
    "concentrate on, focus on, think only about": "buS",
    "classification": "buv",
    "classify": "buv",
    "sergeant": "bu'",
    "torpedoes": "cha",
    "emergency drop": "chagh",
    "they, them (capable of using language)": "chaH",
    "sky": "chal",
    "technician": "chamwI'",
    "back (of hand)": "chap",
    "perhaps": "chaq",
    "conquer": "chargh",
    "achieve": "chav",
    "achievement": "chav",
    "allow, permit": "chaw'",
    "how?": "chay'",
    "two": "cha'",
    "second": "cha'DIch",
    "day after tomorrow": "cha'Hu'",
    "day before yesterday": "cha'leS",
    "twice": "cha'logh",
    "dilithium": "cha'puj",
    "dilithium crystal": "cha'pujqut",
    "be drunk, intoxicated": "chech",
    "return": "chegh",
    "defect": "cheH",
    "add": "chel",
    "build up, take form": "chen",
    "prosper, be prosperous": "chep",
    "establish, set up": "cher",
    "tolerate": "chergh",
    "torpedo tube": "chetvI'",
    "separate": "chev",
    "rule, reign, run": "che'",
    "purposely, on purpose, intentionally": "chIch",
    "admit": "chIj",
    "navigate": "chIj",
    "navigator": "chIjwI'",
    "be empty, deserted, uninhabited": "chIm",
    "cut, trim (hair)": "chIp",
    "temple (structure)": "chIrgh",
    "be white": "chIS",
    "alter, change": "choH",
    "change": "choH",
    "close in, get closer, come nearer": "chol",
    "ponytail holder": "choljaH",
    "bartender": "chom",
    "be vertical": "chong",
    "bite": "chop",
    "preserve": "choq",
    "belly": "chor",
    "eight": "chorgh",
    "eighth": "chorghDIch",
    "dirt under fingernails": "choS",
    "twilight": "choS",
    "murder": "chot",
    "specimen": "chovnatlh",
    "ice": "chuch",
    "be innocent": "chun",
    "meteor": "chunDab",
    "accelerate": "chung",
    "recommend, suggest": "chup",
    "range, distance": "chuq",
    "nobility": "chuQun",
    "be noisy": "chuS",
    "law": "chut",
    "be left over": "chuv",
    "leftovers (grammatical term)": "chuvmey",
    "thrusters": "chuyDaH",
    "be new": "chu'",
    "engage, activate (a device)": "chu'",
    "be absent": "Dach",
    "now": "DaH",
    "be interesting": "Daj",
    "be boring": "Dal",
    "occupy (military term)": "Dan",
    "nonsense": "Dap",
    "eavesdrop": "Daq",
    "ponytail": "DaQ",
    "boot": "DaS",
    "boot spike": "DaSpu'",
    "everywhere": "Dat",
    "revolt": "Daw'",
    "revolt, revolution": "Daw'",
    "corporal (rank)": "Da'",
    "desert": "Deb",
    "surround": "Dech",
    "helm": "Degh",
    "helmsman": "DeghwI'",
    "collapse": "Dej",
    "describe": "Del",
    "Denebia": "DenIb",
    "Denebian": "DenIbngan",
    "Denebian slime devil": "DenIb Qatlh",
    "being (nonhumanoid)": "Dep",
    "credit (monetary unit)": "DeQ",
    "arm (body part)": "DeS",
    "lead, guide": "Dev",
    "data, information": "De'",
    "computer": "De'wI'",
    "privilege": "DIb",
    "certainty": "DIch",
    "pay for": "DIl",
    "trillium": "DIlyum",
    "spin": "DIng",
    "noun": "DIp",
    "skin": "DIr",
    "cave": "DIS",
    "confess": "DIS",
    "year (Klingon)": "DIS",
    "be guilty": "DIv",
    "federation, organization": "DIvI'",
    "Federation battle cruiser": "DIvI'may'Duj",
    "velocity": "Do",
    "be rude": "Doch",
    "thing": "Doch",
    "be foolish, silly": "Dogh",
    "unconditional surrender": "Doghjey",
    "back away from, back off, get away from": "DoH",
    "be impressive": "Doj",
    "entity": "Dol",
    "radan (crude dilithium crystal)": "Dom",
    "be parallel, go parallel to": "Don",
    "side": "Dop",
    "be orange, red": "Doq",
    "claim (territory)": "DoQ",
    "escort": "Dor",
    "target": "DoS",
    "status": "Dotlh",
    "be tired": "Doy'",
    "Troyius": "Doy'yuS",
    "be fortunate, lucky": "Do'",
    "luckily, with luck": "Do'",
    "back (of body)": "Dub",
    "improve": "Dub",
    "mix": "DuD",
    "be vigilant": "Dugh",
    "be possible": "DuH",
    "possibility, option": "DuH",
    "instincts": "Duj",
    "ship, vessel": "Duj",
    "nap": "Dum",
    "be wonderful, great": "Dun",
    "area above, area overhead": "Dung",
    "strategy": "Dup",
    "stab": "DuQ",
    "torpedo tube": "DuS",
    "school": "DuSaQ",
    "advance": "Duv",
    "agent, emissary": "Duy",
    "be defective": "Duy'",
    "defect": "Duy'",
    "farm": "Du'",
    "gargle": "ghagh",
    "he, she, him, her": "ghaH",
    "have, possess": "ghaj",
    "or, either/or (joining nouns)": "ghap",
    "contribute": "ghaq",
    "conduct diplomacy": "ghar",
    "diplomacy": "ghar",
    "serpent, worm": "ghargh",
    "diplomat": "gharwI'",
    "dominate": "ghatlh",
    "be rough": "ghegh",
    "midnight snack": "ghem",
    "consent": "ghIb",
    "nose": "ghIch",
    "necklace": "ghIgh",
    "scare": "ghIH",
    "be messy, sloppy": "ghIj",
    "exile": "ghIm",
    "court-martial": "ghIpDIj",
    "vacation, take a vacation": "ghIQ",
    "descend": "ghIr",
    "manuscript": "ghItlh",
    "write": "ghItlh",
    "circle": "gho",
    "ethics": "ghob",
    "no (answer to a question)": "ghobe'",
    "destination": "ghoch",
    "stuff": "ghoD",
    "voice": "ghogh",
    "argue, dispute": "ghoH",
    "learn": "ghoj",
    "teach, instruct": "ghojmoH",
    "student": "ghojwI'",
    "opponent, adversary": "ghol",
    "group, party": "ghom",
    "meet, encounter, assemble, rendezvous": "ghom",
    "scatter, disperse": "ghomHa'",
    "crowd": "ghom'a'",
    "abuse": "ghong",
    "hand": "ghop",
    "asteroid": "ghopDap",
    "spy": "ghoq",
    "spy": "ghoqwI'",
    "break": "ghor",
    "surface (of a planet)": "ghor",
    "when?": "ghorgh",
    "approach, go away from, proceed, come, follow (a course)": "ghoS",
    "thrust": "ghoS",
    "person (humanoid)": "ghot",
    "recognize": "ghov",
    "step on": "gho'",
    "sublight speed": "gho'Do",
    "baby": "ghu",
    "alert": "ghuH",
    "prepare for, be alerted to": "ghuH",
    "alert, warn": "ghuHmoH",
    "alarm": "ghum",
    "alarm, sound an alarm": "ghum",
    "program (a computer)": "ghun",
    "be hungry": "ghung",
    "swallow": "ghup",
    "increase": "ghur",
    "be prepared, ready (to launch)": "ghuS",
    "situation": "ghu'",
    "be smooth": "Hab",
    "data transceiving device": "HablI'",
    "be developed (e.g., civilization)": "Hach",
    "study": "HaD",
    "laugh": "Hagh",
    "dread": "Haj",
    "source": "Hal",
    "nacelle": "HanDogh",
    "matter": "Hap",
    "surgery": "Haq",
    "saccharin": "HaQchor",
    "believe": "Har",
    "visual display": "HaSta",
    "be illegal": "Hat",
    "temperature": "Hat",
    "country, countryside": "Hatlh",
    "flee, get out": "Haw'",
    "duel": "Hay'",
    "let's go, come on": "Ha'",
    "animal": "Ha'DIbaH",
    "course, route": "He",
    "intend, mean to": "Hech",
    "retreat": "HeD",
    "parallel course": "HeDon",
    "die": "Hegh",
    "be fatal": "HeghmoH",
    "edge": "HeH",
    "rob": "Hej",
    "be proud": "Hem",
    "comply": "HeQ",
    "medicine": "Hergh",
    "commit a crime": "HeS",
    "crime": "HeS",
    "criminal": "HeSwI'",
    "receive": "Hev",
    "smell, emit odor": "He'",
    "stink": "He'sSo'",
    "handgun": "HIch",
    "airlock": "HIchDal",
    "menu": "HIDjolev",
    "fight dirty": "HIgh",
    "yes, true (answer to yes/no question)": "HIja'",
    "uniform": "HIp",
    "liquor": "HIq",
    "attack": "HIv",
    "glass (tumbler)": "HIvje'",
    "dictator": "HI'",
    "dictatorship": "HI'tuy",
    "yawn": "Hob",
    "everyone, all, everything": "Hoch",
    "captain": "HoD",
    "week (Klingon)": "Hogh",
    "kill": "HoH",
    "commit suicide": "HoH'egh",
    "be cautious": "Hoj",
    "language": "Hol",
    "bone": "Hom",
    "doubt": "Hon",
    "impulse power": "Hong",
    "be remote, far": "Hop",
    "expedition": "Hoq",
    "tricorder": "Hoqra'",
    "be strong": "HoS",
    "strength, energy, power": "HoS",
    "energy field": "HoSchem",
    "energy beings": "HoSDo'",
    "be powerful": "HoSghaj",
    "touch, feel": "Hot",
    "project, put on (screen)": "Hotlh",
    "scan": "Hotlh",
    "scanner": "HotlhwI'",
    "star": "Hov",
    "star system": "Hovtay'",
    "congratulate": "Hoy'",
    "admire": "Ho'",
    "tooth": "Ho'",
    "toothache": "Ho''oy'",
    "zoo": "Hu",
    "defend": "Hub",
    "defense": "Hub",
    "money": "Huch",
    "mountain, hill": "HuD",
    "throat": "Hugh",
    "be strange": "Huj",
    "charge (up)": "Huj",
    "be sticky": "Hum",
    "human": "Human",
    "security": "Hung",
    "punish": "Hup",
    "transact": "Huq",
    "outside": "Hur",
    "be dark": "Hurgh",
    "pickle (cucumber)": "Hurgh",
    "hang": "HuS",
    "nine": "Hut",
    "ninth": "HutDIch",
    "be clear, not obstructed": "Huv",
    "eyebrow": "Huy'",
    "days ago": "Hu'",
    "get up": "Hu'",
    "serve (food)": "jab",
    "data transmission": "jabbI'ID",
    "scream, cry out, shout, yell": "jach",
    "enemy": "jagh",
    "go": "jaH",
    "day (from dawn to dawn)": "jaj",
    "dawn": "jajlo'",
    "device": "jan",
    "answer, reply": "jang",
    "be bold": "jaq",
    "month (Klingon)": "jar",
    "tongue": "jat",
    "say": "jatlh",
    "six": "jav",
    "sixth": "javDIch",
    "chat": "jaw",
    "lord": "jaw",
    "tell, report": "ja'",
    "discuss, confer": "ja'chuq",
    "also, and (joining nouns)": "je",
    "disguise": "jech",
    "surrender, give up": "jegh",
    "be absentminded": "jeH",
    "be sharp": "jej",
    "be high": "jen",
    "be self-confident": "jeQ",
    "participate": "jeS",
    "storm": "jev",
    "defeat": "jey",
    "buy, purchase": "je'",
    "feed (someone else)": "je'",
    "hair (on head)": "jIb",
    "I, me": "jIH",
    "viewing screen": "jIH",
    "cooperate": "jIj",
    "neighbor": "jIl",
    "project": "jInmol",
    "penalty": "jIp",
    "be ignorant": "jIv",
    "resources": "jo",
    "be harmful": "joch",
    "stoop": "joD",
    "lord": "joH",
    "area between": "joj",
    "consul": "jojlu'",
    "beam (aboard)": "jol",
    "transport beam": "jol",
    "transport room": "jolpa'",
    "transporter ionizer unit": "jolvoy'",
    "capture": "jon",
    "engine": "jonta'",
    "engineer": "jonwI'",
    "flap, flutter, wave": "joq",
    "or, and/or (joining nouns)": "joq",
    "flag": "joqwI'",
    "explode": "jor",
    "explosive": "jorwI'",
    "gossip": "joS",
    "rumor, gossip": "joS",
    "be calm": "jot",
    "be uneasy": "jotHa'",
    "take down": "jotlh",
    "torture": "joy'",
    "machinery": "jo'",
    "be immortal": "jub",
    "be mortal": "jubbe'",
    "home": "juH",
    "be odd": "jum",
    "evade, take evasive action": "jun",
    "friend": "jup",
    "overtake, pass": "juS",
    "measure": "juv",
    "transmit data (away from a place)": "lab",
    "exaggerate": "lach",
    "read": "laD",
    "ability": "laH",
    "accept": "laj",
    "acceptance": "laj",
    "religion": "lalDan",
    "be dirty": "lam",
    "dirt": "lam",
    "place": "lan",
    "be thin": "lang",
    "fire, energize (e.g., thrusters)": "laQ",
    "smell, sense odors": "largh",
    "factory": "laSvargh",
    "be many": "law'",
    "promise": "lay'",
    "commander": "la'",
    "see": "legh",
    "maintain": "leH",
    "maintenance": "leH",
    "get out, take out": "lel",
    "roam, travel, rove": "leng",
    "trip, voyage": "leng",
    "rover (grammatical term)": "lengwI'",
    "switch": "leQ",
    "days from now": "leS",
    "rest, relax": "leS",
    "shore leave": "leSpoH",
    "be hard (like a rock)": "let",
    "be special, exceptional": "le'",
    "ride": "lIgh",
    "introduce": "lIH",
    "forget": "lIj",
    "panic": "lIm",
    "espionage": "lInDab",
    "produce": "lIng",
    "round up": "lIq",
    "adjust": "lIS",
    "comet": "lIy",
    "be useful": "lI'",
    "transmit data (to a place)": "lI'",
    "obey": "lob",
    "disobey": "lobHa'",
    "mustache": "loch",
    "male, man": "loD",
    "boy": "loDHom",
    "husband": "loDnal",
    "brother": "loDnI'",
    "space": "logh",
    "administer": "loH",
    "administration": "loH",
    "be all gone": "loj",
    "door, gate": "lojmIt",
    "attitude-control thrusters": "lolSeHcha",
    "corpse": "lom",
    "abandon": "lon",
    "celebrate": "lop",
    "slightly, a little bit": "loQ",
    "four": "loS",
    "wait (for)": "loS",
    "fourth": "loSDIch",
    "quadrotriticale": "loSpev",
    "catastrophe": "lot",
    "rebel": "lotlh",
    "rebel": "lotlhwI'",
    "guess": "loy",
    "use": "lo'",
    "be valuable": "lo'laH",
    "be worthless": "lo'laHbe'",
    "equipment, gear": "luch",
    "be right, correct": "lugh",
    "yank": "luH",
    "fail": "luj",
    "refuge": "lulIgh",
    "postpone, procrastinate": "lum",
    "second (of time)": "lup",
    "transport": "lup",
    "yes, okay, I will": "luq",
    "pupil (of eye)": "lur",
    "tradition": "lurDech",
    "story": "lut",
    "be primitive": "lutlh",
    "treaty": "mab",
    "be small": "mach",
    "betray": "magh",
    "traitor": "maghwI'",
    "ten (number-forming element)": "maH",
    "we, us": "maH",
    "good (expressing satisfaction)": "maj",
    "well done, very good": "majQa'",
    "business": "malja'",
    "soldier": "mang",
    "army": "mangghom",
    "proclaim": "maq",
    "moon": "maS",
    "prefer": "maS",
    "moonlight": "maSwov",
    "paper clip": "mavjop",
    "offend": "maw",
    "be crazy": "maw'",
    "be fair": "may",
    "battle": "may'",
    "battle cruiser": "may'Duj",
    "battle array": "may'morgh",
    "accommodate": "ma'",
    "guest": "meb",
    "trade": "mech",
    "lunch": "megh",
    "bridge (of a ship)": "meH",
    "catalog": "mem",
    "plastic": "mep",
    "reason": "meq",
    "burn": "meQ",
    "surprise": "mer",
    "stop, cease": "mev",
    "sector, zone": "mIch",
    "colony": "mID",
    "be evil": "mIgh",
    "delay": "mIm",
    "eye": "mIn",
    "be rich": "mIp",
    "machinery": "mIqta'",
    "chain": "mIr",
    "be confused, mixed up": "mIS",
    "confusion": "mIS",
    "confuse": "mISmoH",
    "helmet": "mIv",
    "brag": "mIy",
    "number": "mI'",
    "be alone": "mob",
    "superior": "moch",
    "hurry": "moD",
    "be frustrated": "mogh",
    "be ugly": "moH",
    "prefix": "moHaq",
    "become": "moj",
    "suffix": "mojaq",
    "bury": "mol",
    "grave": "mol",
    "capital (of a place)": "mon",
    "neck": "mong",
    "collar": "mongDech",
    "robe": "mop",
    "sphere": "moQ",
    "compromise": "moS",
    "be usual": "motlh",
    "be unusual": "motlhbe'",
    "slingshot": "moy'bI'",
    "cage": "mo'",
    "be legal": "mub",
    "present": "much",
    "presentation": "much",
    "atmosphere": "muD",
    "translate": "mugh",
    "mugato": "mughato'",
    "translator": "mughwI'",
    "execute, put to death": "muH",
    "be wrong": "muj",
    "be stubborn": "mul",
    "origin": "mung",
    "impact, strike": "mup",
    "hate, detest": "muS",
    "be selfish": "mut",
    "species": "mut",
    "join": "muv",
    "word": "mu'",
    "dictionary": "mu'ghom",
    "vocabulary": "mu'tay'",
    "sentence": "mu'tlhegh",
    "plan": "nab",
    "head": "nach",
    "here, hereabouts": "naDev",
    "rock, stone": "nagh",
    "dream": "naj",
    "be simple": "nap",
    "appear": "nargh",
    "be vicious": "naS",
    "paper": "nav",
    "squadron": "nawlogh",
    "access": "naw'",
    "marry (wife does this)": "nay",
    "course, dish (at a meal)": "nay'",
    "be lateral, move laterally": "nech",
    "soldiers": "negh",
    "only, merely, just": "neH",
    "want": "neH",
    "neutral zone": "neHmaH",
    "look for, seek, search for": "nej",
    "years from now": "nem",
    "lie, fib": "nep",
    "that (previous topic)": "net",
    "ten thousand": "netlh",
    "yeoman": "ne'",
    "be identical": "nIb",
    "ammunition": "nIch",
    "attempt, try": "nID",
    "right (side)": "nIH",
    "steal": "nIH",
    "thief": "nIHwI'",
    "leak": "nIj",
    "fuel": "nIn",
    "breakfast": "nIQ",
    "hinder, interfere": "nIS",
    "finger": "nItlh",
    "be superior": "nIv",
    "pajamas": "nIvnav",
    "be long, lengthy (duration)": "nI'",
    "gift": "nob",
    "give": "nob",
    "sensor": "noch",
    "retaliate": "noD",
    "writhe": "nogh",
    "judge, estimate": "noH",
    "lend": "noj",
    "funeral": "nol",
    "fast, quickly": "nom",
    "be rotten": "non",
    "be passionate": "nong",
    "omit": "nop",
    "deodorant": "noSvagh",
    "never": "not",
    "alien, foreigner": "nov",
    "be foreign, alien": "nov",
    "be famous, well known": "noy",
    "ancestors": "no'",
    "be suspect": "nub",
    "predecessor": "nubwI'",
    "coward": "nuch",
    "examine": "nuD",
    "society": "nugh",
    "weapon": "nuH",
    "small arms": "nuHHom",
    "mouth": "nuj",
    "promote": "num",
    "precede": "nung",
    "decrease": "nup",
    "Neural": "nural",
    "Neuralese": "nuralngan",
    "what?": "nuq",
    "where?": "nuqDaq",
    "what do you want? (greeting)": "nuqneH",
    "annoy, bother": "nuQ",
    "dignity": "nur",
    "ridicule": "nuS",
    "person (humanoid)": "nuv",
    "disappear, vanish": "ngab",
    "debate": "ngach",
    "inhabitant": "ngan",
    "support (military term)": "ngaq",
    "contain (have inside)": "ngaS",
    "gunpowder": "ngat",
    "writer's cramp": "ngav",
    "be counterfeit, false, fake": "ngeb",
    "valley": "ngech",
    "be easy": "ngeD",
    "send": "ngeH",
    "infect": "ngej",
    "forest, woods": "ngem",
    "lake": "ngeng",
    "override": "ngep",
    "theory": "nger",
    "sell": "ngev",
    "take away": "nge'",
    "dare": "ngIl",
    "be putrid": "ngIm",
    "borrow": "ngIp",
    "patrol": "ngIv",
    "fact": "ngoD",
    "smear": "ngoH",
    "be restless": "ngoj",
    "experiment": "ngong",
    "code": "ngoq",
    "goal": "ngoQ",
    "cheat": "ngor",
    "dissolve": "ngoS",
    "be fanatical": "ngotlh",
    "be responsible": "ngoy'",
    "be old (not new)": "ngo'",
    "cape (clothing)": "ngup",
    "identify": "ngu'",
    "follow (rules)": "pab",
    "grammar": "pab",
    "nothing, none": "pagh",
    "or, either/or (joining sentences)": "pagh",
    "zero": "pagh",
    "gown": "paH",
    "resign": "paj",
    "book": "paq",
    "teachings": "paQDI'norgh",
    "dislike": "par",
    "like": "parHa'",
    "socks": "paSlogh",
    "system": "pat",
    "be urgent": "pav",
    "arrive": "paw",
    "collide": "paw'",
    "regret": "pay",
    "suddenly": "pay'",
    "room": "pa'",
    "there, over there, thereabouts": "pa'",
    "snow": "peD",
    "keep something secret": "pegh",
    "secret": "pegh",
    "demolish": "pej",
    "daytime": "pem",
    "midday": "pemjep",
    "torpedo": "peng",
    "raise": "pep",
    "label": "per",
    "acid": "pey",
    "cut": "pe'",
    "blame": "pIch",
    "fault, blame": "pIch",
    "ruins": "pIgh",
    "expect": "pIH",
    "be suspicious": "pIH",
    "often": "pIj",
    "be different": "pIm",
    "boss": "pIn",
    "spine": "pIp",
    "Klingon writing system": "pIqaD",
    "done": "pItlh",
    "be healthy": "pIv",
    "warp drive": "pIvghor",
    "warp factor": "pIvlob",
    "be fat": "pI'",
    "morning": "po",
    "hair (on body)": "pob",
    "plant": "poch",
    "be clipped": "poD",
    "glove": "pogh",
    "period of time": "poH",
    "time": "poH",
    "analysis": "poj",
    "analyze": "poj",
    "keep, save": "pol",
    "dysentery": "pom",
    "persuade, convince": "pon",
    "name": "pong",
    "name, call": "pong",
    "indigestion": "poq",
    "demand, require": "poQ",
    "body": "porgh",
    "be open, opened": "poS",
    "left (side)": "poS",
    "open": "poSmoH",
    "consequential thing, something important": "potlh",
    "afternoon": "pov",
    "be excellent": "pov",
    "be expert, skilled": "po'",
    "boil": "pub",
    "toilet": "puch",
    "washroom": "puchpa'",
    "dregs": "pugh",
    "land": "puH",
    "be weak": "puj",
    "weaken": "pujmoH",
    "weakling": "pujwI'",
    "accusation": "pum",
    "accuse": "pum",
    "fall": "pum",
    "mercy": "pung",
    "be perfect, exact": "pup",
    "kick": "pup",
    "child, offspring": "puq",
    "daughter": "puqbe'",
    "son": "puqloD",
    "be fed up": "puQ",
    "be few, be several, be a handful": "puS",
    "sight (with gunsight)": "puS",
    "fly": "puv",
    "wreck": "puy",
    "nova": "puyjaq",
    "phaser": "pu'",
    "phaser rifle": "pu'beH",
    "phaser crew": "pu'beq",
    "phaser banks": "pu'DaH",
    "phaser pistol": "pu'HIch",
    "be bad": "qab",
    "face": "qab",
    "building, structure": "qach",
    "interrupt": "qagh",
    "sir": "qaH",
    "be corrupt": "qal",
    "corrupt": "qalmoH",
    "foot": "qam",
    "prisoner": "qama'",
    "be old (not young)": "qan",
    "insist": "qap",
    "occur, happen": "qaS",
    "wrap": "qat",
    "why?": "qatlh",
    "remember": "qaw",
    "memory banks": "qawHaq",
    "remind": "qawmoH",
    "be a problem, be a hassle": "qay'",
    "Genesis": "qa'vam",
    "idea": "qech",
    "vacate": "qeD",
    "resent": "qeH",
    "be grouchy, mean": "qej",
    "kellicam": "qelI'qam",
    "bring": "qem",
    "ancestor": "qempa'",
    "carry, convey": "qeng",
    "meeting": "qep",
    "drill (military)": "qeq",
    "practice, train, prepare": "qeq",
    "advice": "qeS",
    "advise": "qeS",
    "run, jog": "qet",
    "be dull, uninteresting": "qetlh",
    "crowd": "qev",
    "kevas": "qevaS",
    "cheek": "qevpob",
    "galaxy": "qIb",
    "galactic rim": "qIbHeS",
    "condemn": "qIch",
    "shortcut": "qIgh",
    "meet (for the first time)": "qIH",
    "be black": "qIj",
    "cancel": "qIl",
    "pay attention, concentrate": "qIm",
    "disregard": "qImHa'",
    "hit (with hand, fist, implement)": "qIp",
    "mutiny": "qIQ",
    "knee": "qIv",
    "sign (a treaty)": "qI'",
    "partner": "qoch",
    "belt": "qogh",
    "fool": "qoH",
    "cliff": "qoj",
    "or, and/or (joining sentences)": "qoj",
    "record": "qon",
    "arrest": "qop",
    "robot": "qoq",
    "scavenge": "qor",
    "family": "qorDu'",
    "tickle": "qotlh",
    "birthday": "qoS",
    "plead, beg": "qoy'",
    "be rare": "qub",
    "kidnap": "quch",
    "cruise": "qugh",
    "dandruff": "quHvaj",
    "fire": "qul",
    "govern": "qum",
    "government": "qum",
    "history": "qun",
    "scold": "qun",
    "elder": "qup",
    "Council of Elders": "quprIp",
    "happen simultaneously": "quq",
    "be greedy": "qur",
    "chair": "quS",
    "crystal (geologic formation)": "qut",
    "be fierce": "qu'",
    "theragen": "Qab",
    "be dry": "QaD",
    "error, mistake": "Qagh",
    "err, be mistaken, make a mistake": "Qagh",
    "help, aid": "QaH",
    "stand": "Qam",
    "protect": "Qan",
    "work, function, succeed": "Qap",
    "success": "Qapla'",
    "be good": "QaQ",
    "fissure": "Qargh",
    "troops": "QaS",
    "be popular": "Qat",
    "be difficult": "Qatlh",
    "be final, last": "Qav",
    "destroy": "Qaw'",
    "transfer": "Qay",
    "blow one's top": "Qay'",
    "type of animal": "Qa'",
    "ring (for finger)": "Qeb",
    "science": "QeD",
    "science officer": "QeDpIn",
    "anger": "QeH",
    "be angry, mad": "QeH",
    "doctor, physician": "Qel",
    "be tight": "Qey",
    "be loose": "QeyHa'",
    "loosen": "QeyHa'moH",
    "tighten": "QeymoH",
    "restaurant": "Qe'",
    "Klingon agonizer": "QIghpej",
    "shadow": "QIb",
    "speech (vocal sounds)": "QIch",
    "wound": "QID",
    "damage, cause damage": "QIH",
    "damage, destruction": "QIH",
    "explain": "QIj",
    "be stupid": "QIp",
    "slowly": "QIt",
    "be inferior": "QIv",
    "military": "QI'",
    "be dangerous": "Qob",
    "danger": "Qob",
    "disagree": "Qoch",
    "agree": "Qochbe'",
    "maneuver (engines)": "QoD",
    "type of animal": "Qogh",
    "make war": "Qoj",
    "experience an earthquake or tremor": "Qom",
    "journal, log": "QonoS",
    "sleep": "Qong",
    "be worn out": "Qop",
    "wear out": "QopmoH",
    "take care of, care for": "Qorgh",
    "be sorry": "QoS",
    "lie, recline": "Qot",
    "disable": "Qotlh",
    "hear": "Qoy",
    "no, I won't, I refuse": "Qo'",
    "think": "Qub",
    "be happy": "Quch",
    "forehead": "Quch",
    "be unhappy": "QuchHa'",
    "insurrection": "QuD",
    "disaster": "Qugh",
    "game": "Quj",
    "play a game": "Quj",
    "research": "Qul",
    "communicate": "Qum",
    "communications officer": "QumpIn",
    "communicator, communications device": "QumwI'",
    "be young": "Qup",
    "engine": "QuQ",
    "conspiracy": "QuS",
    "conspire": "QuS",
    "be vulgar": "Qut",
    "coordinates": "Quv",
    "duty, quest, mission, task, chore": "Qu'",
    "force, compel": "raD",
    "decay": "ragh",
    "be violent": "ral",
    "be trivial, trifling, unimportant": "ram",
    "night": "ram",
    "midnight": "ramjep",
    "be the same": "rap",
    "connect": "rar",
    "camp (military term)": "raQ",
    "passenger": "raQpo'",
    "remain": "ratlh",
    "floor": "rav",
    "targets": "ray'",
    "order, command": "ra'",
    "commander": "ra'wI'",
    "Regulus": "reghuluS",
    "Regulan": "reghuluSngan",
    "Regulan bloodworm": "reghuluS 'Iwghargh",
    "always": "reH",
    "play": "reH",
    "worrywart": "rejmorgh",
    "hour": "rep",
    "area beside, area next to": "retlh",
    "citizen": "rewbe'",
    "be lame": "rIgh",
    "energize": "rIH",
    "energizer": "rIHwI'",
    "be accomplished, finished": "rIn",
    "council, assembly": "rIp",
    "be injured": "rIQ",
    "injure": "rIQmoH",
    "embassy": "rIvSo'",
    "Remus": "rIymuS",
    "trunk (of body)": "ro",
    "population": "roghvaH",
    "make peace": "roj",
    "peace": "roj",
    "truce": "rojHom",
    "peace treaty": "rojmab",
    "beard": "rol",
    "accord": "rom",
    "Romulus": "romuluS",
    "Romulan": "romuluSngan",
    "be sick, ill": "rop",
    "disease": "rop",
    "infirmary": "ropyaH",
    "put down": "roQ",
    "be fat": "ror",
    "be tough": "rotlh",
    "fist": "ro'",
    "antimatter": "rugh",
    "control manually, by hand": "ruQ",
    "fine, tax": "rup",
    "resemble": "rur",
    "sometimes": "rut",
    "justice": "ruv",
    "be temporary": "ru'",
    "decline, deteriorate": "Sab",
    "expand": "Sach",
    "thousand": "SaD",
    "be serious": "Sagh",
    "be present (not absent)": "SaH",
    "care (about), be concerned (about)": "SaH",
    "pet": "Saj",
    "ascend": "Sal",
    "fate": "San",
    "obliterate": "Sang",
    "volunteer": "Sap",
    "land": "Saq",
    "landing party": "Saqghom",
    "cry": "SaQ",
    "be varied, various": "Sar",
    "variety": "Sar",
    "agriculture": "Satlh",
    "be horizontal": "SaS",
    "marry (husband does this)": "Saw",
    "be clean": "Say'",
    "general (rank)": "Sa'",
    "race (type, sort, class)": "Segh",
    "control": "SeH",
    "control panel": "SeHlaw",
    "cause trouble": "Seng",
    "trouble": "Seng",
    "breed": "Sep",
    "region": "Sep",
    "fault (seismic)": "Seq",
    "progress": "Ser",
    "Sherman's Planet": "SermanyuQ",
    "steam": "SeS",
    "bandage": "Sev",
    "contain (an enemy)": "Sev",
    "be excited": "Sey",
    "excite": "SeymoH",
    "frequency (radio)": "Se'",
    "satellite": "SIbDoH",
    "reach": "SIch",
    "patient": "SID",
    "influence": "SIgh",
    "bend": "SIH",
    "slit": "SIj",
    "calculate": "SIm",
    "gas": "SIp",
    "endure, bear": "SIQ",
    "rain": "SIS",
    "wonder": "SIv",
    "seven": "Soch",
    "seventh": "SochDIch",
    "flood": "SoD",
    "you": "SoH",
    "quarrel": "Sol",
    "hull": "Som",
    "muscle": "Somraw",
    "relieve": "Son",
    "eat": "Sop",
    "be closed, shut": "SoQ",
    "speech, lecture, address": "SoQ",
    "close, shut": "SoQmoH",
    "tree": "Sor",
    "sabotage": "Sorgh",
    "mother": "SoS",
    "grandmother": "SoSnI'",
    "be distressed, be in distress": "Sot",
    "distress call": "Sotlaw'",
    "know": "Sov",
    "be clumsy": "Soy'",
    "hide, cloak": "So'",
    "cloaking device": "So'wI'",
    "be solid": "Sub",
    "visit": "Such",
    "be green, blue, yellow": "SuD",
    "ready, standing by": "SuH",
    "disturb": "Suj",
    "discipline": "Sun",
    "native": "Sung",
    "jump": "Sup",
    "resource": "Sup",
    "acquire, obtain, get": "Suq",
    "be toxic": "SuQ",
    "force field": "Surchem",
    "skin": "Surgh",
    "wind, breeze": "SuS",
    "clothing": "Sut",
    "negotiate": "Sutlh",
    "fight": "Suv",
    "merchant": "Suy",
    "merchant ship": "SuyDuj",
    "record": "ta",
    "bar, saloon, cocktail lounge": "tach",
    "be frozen": "taD",
    "freeze": "taDmoH",
    "be at a negative angle": "taH",
    "knife, dagger": "taj",
    "be quiet": "tam",
    "exchange, substitute": "tam",
    "silence": "tammoH",
    "be weird": "taQ",
    "exhaust": "taQbang",
    "poison": "tar",
    "solution (liquid)": "taS",
    "ion": "tat",
    "be civilized": "tay",
    "civilize": "taymoH",
    "civilization": "tayqeq",
    "be together": "tay'",
    "accomplish": "ta'",
    "accomplishment": "ta'",
    "emperor": "ta'",
    "fill": "teb",
    "jurisdiction": "teblaw'",
    "be true": "teH",
    "scientist": "tej",
    "wing": "tel",
    "Tellun Star System": "telun Hovtay'",
    "deny": "tem",
    "embark": "ten",
    "space station": "tengchaH",
    "cargo": "tep",
    "cargo carrier": "tepqengwI'",
    "remove, take off": "teq",
    "Earth": "tera'",
    "Terran, Earther": "tera'ngan",
    "melt": "tet",
    "prize": "tev",
    "confide": "tey'",
    "vegetation": "tI",
    "insult": "tIch",
    "custom": "tIgh",
    "ray": "tIH",
    "board, go aboard": "tIj",
    "be big": "tIn",
    "be long, lengthy (of an object)": "tIq",
    "heart": "tIq",
    "be ancient": "tIQ",
    "grain": "tIr",
    "be light (weight)": "tIS",
    "enjoy": "tIv",
    "fix, repair": "tI'",
    "prove": "tob",
    "palm (of hand)": "toch",
    "save, rescue": "toD",
    "count": "togh",
    "so, well": "toH",
    "deceive, trick": "toj",
    "freighter": "tongDuj",
    "topaline": "toplIn",
    "be inhabited": "toq",
    "kneel": "tor",
    "climb": "toS",
    "serve (a master)": "toy'",
    "servant": "toy'wI'",
    "tactics": "to'",
    "forbid": "tuch",
    "soon": "tugh",
    "be ashamed": "tuH",
    "maneuver (military term)": "tuH",
    "shame": "tuHmoH",
    "be hot": "tuj",
    "heat": "tuj",
    "hope": "tul",
    "agency": "tum",
    "be soft": "tun",
    "discourage": "tung",
    "encourage": "tungHa'",
    "minute (of time)": "tup",
    "wear (clothes)": "tuQ",
    "mind sifter (Klingon psychic probe)": "tuQDoq",
    "undress": "tuQHa'moH",
    "put on (clothes)": "tuQmoH",
    "column": "tut",
    "be patient": "tuv",
    "discover, find, observe, notice": "tu'",
    "be free, independent": "tlhab",
    "freedom, independence": "tlhab",
    "take": "tlhap",
    "chronometer": "tlhaq",
    "be funny": "tlhaQ",
    "chase, follow": "tlha'",
    "depart": "tlheD",
    "line, rope": "tlhegh",
    "accompany": "tlhej",
    "progress": "tlhetlh",
    "turn": "tlhe'",
    "be incompetent": "tlhIb",
    "smoke": "tlhIch",
    "you (plural)": "tlhIH",
    "mine": "tlhIl",
    "mineral": "tlhIl",
    "miner": "tlhIlwI'",
    "Klingon": "tlhIngan",
    "Klingon Empire": "tlhIngan wo'",
    "be insubordinate": "tlhIv",
    "ask": "tlhob",
    "contradict": "tlhoch",
    "marriage": "tlhogh",
    "realize": "tlhoj",
    "be raw, unprocessed": "tlhol",
    "nostril": "tlhon",
    "barter, bargain": "tlhong",
    "conglomeration": "tlhoQ",
    "wheeze": "tlhov",
    "exhaust": "tlhuch",
    "breath": "tlhuH",
    "breathe": "tlhuH",
    "whisper": "tlhup",
    "drink": "tlhutlh",
    "be tempted": "tlhu'",
    "tempt": "tlhu'moH",
    "five": "vagh",
    "fifth": "vaghDIch",
    "holster": "vaH",
    "so, then, thus, in that case": "vaj",
    "be clever, smart, intelligent": "val",
    "salute": "van",
    "act, take action": "vang",
    "be aggressive": "vaQ",
    "hundred": "vatlh",
    "percent": "vatlhvI'",
    "father": "vav",
    "grandfather": "vavnI'",
    "somebody, something, anybody, anything": "vay'",
    "boundary": "veH",
    "wake up, cease sleeping": "vem",
    "wake (someone) up": "vemmoH",
    "city": "veng",
    "village": "vengHom",
    "garbage": "veQ",
    "garbage scow": "veQDuj",
    "dock": "vergh",
    "war": "veS",
    "cockroach": "vetlh",
    "thruster": "vIj",
    "whine": "vIng",
    "tell the truth": "vIt",
    "accumulate": "vI'",
    "emperor": "voDleH",
    "somewhere": "vogh",
    "ransom": "voHDajbo'",
    "shoulder": "volchaH",
    "hypnotize": "vong",
    "trust, have faith in": "voq",
    "distrust": "voqHa'",
    "choke": "voQ",
    "cure": "vor",
    "propel": "vo'",
    "hostage": "vub",
    "opinion": "vuD",
    "be unconscious": "vul",
    "Vulcan (planet)": "vulqan",
    "Vulcan (person)": "vulqangan",
    "work, toil": "vum",
    "pity": "vup",
    "fascinate": "vuQ",
    "limit": "vuS",
    "cook": "vut",
    "galley": "vutpa'",
    "respect": "vuv",
    "manage": "vu'",
    "manager": "vu'wI'",
    "hunt": "wam",
    "phenomenon, event, occurrence": "wanI'",
    "shoe": "waq",
    "obstruct": "waQ",
    "be pure": "watlh",
    "divide": "wav",
    "base (military term)": "waw'",
    "one": "wa'",
    "first": "wa'DIch",
    "yesterday": "wa'Hu'",
    "tomorrow": "wa'leS",
    "once": "wa'logh",
    "ten": "wa'maH",
    "tenth": "wa'maHDIch",
    "be disgraced": "web",
    "confine": "wegh",
    "raid": "weH",
    "not yet": "wej",
    "three": "wej",
    "third": "wejDIch",
    "charming (used only ironically)": "wejpuH",
    "violate": "wem",
    "violation": "wem",
    "jacket, coat": "wep",
    "glow": "wew",
    "be sour": "wIb",
    "myth": "wIch",
    "genius": "wIgh",
    "be ruthless": "wIH",
    "farm": "wIj",
    "choice": "wIv",
    "choose, select": "wIv",
    "tactical display": "wIy",
    "throw away": "woD",
    "pick up": "woH",
    "authority, political power": "woQ",
    "chin": "woS",
    "verb": "wot",
    "be light, bright": "wov",
    "empire": "wo'",
    "decide": "wuq",
    "have a headache": "wuQ",
    "lip": "wuS",
    "underground": "wutlh",
    "depend on, rely on": "wuv",
    "tactical officer": "ya",
    "mind, brain": "yab",
    "pet, stroke": "yach",
    "toe": "yaD",
    "duty station, station": "yaH",
    "understand": "yaj",
    "misinterpret": "yajHa'",
    "be enough, sufficient": "yap",
    "officer": "yaS",
    "ground": "yav",
    "victory, triumph": "yay",
    "be shocked, dumbfounded": "yay'",
    "assembly, council": "yej",
    "sin": "yem",
    "be careful": "yep",
    "be careless": "yepHa'",
    "pause": "yev",
    "vent": "yIb",
    "tribble": "yIH",
    "life": "yIn",
    "live": "yIn",
    "life signs": "yInroH",
    "life-support system": "yIntagh",
    "be wet": "yIQ",
    "walk": "yIt",
    "chew": "yIv",
    "tunic": "yIvbeH",
    "harvest": "yob",
    "shield": "yoD",
    "be brave": "yoH",
    "judgment": "yoj",
    "conflict": "yol",
    "be satisfied": "yon",
    "satisfy": "yonmoH",
    "get in": "yong",
    "pants": "yopwaH",
    "humanoid": "yoq",
    "district, area": "yoS",
    "invade": "yot",
    "invasion": "yot",
    "field (of land)": "yotlh",
    "charge (military term)": "yov",
    "be upside down": "yoy",
    "fleet (of ships)": "yo'",
    "chocolate": "yuch",
    "be dishonest": "yuD",
    "be honest": "yuDHa'",
    "festival": "yupma'",
    "planet": "yuQ",
    "planetoid": "yuQHom",
    "United Federation of Planets": "yuQjIjQa'",
    "push": "yuv",
    "question, interrogate": "yu'",
    "wave": "yu'egh",
    "but, nevertheless, even so, however": "'a",
    "but, nevertheless, even so, however": "'ach",
    "vein": "'aD",
    "show, reveal": "'ang",
    "how many? how much?": "'ar",
    "worsen": "'argh",
    "guard": "'av",
    "sting": "'aw'",
    "section": "'ay'",
    "opportunity": "'eb",
    "ready": "'eH",
    "and (joining sentences)": "'ej",
    "starship, starship class": "'ejDo'",
    "Starfleet": "'ejyo'",
    "star base": "'ejyo'waw'",
    "enter, go in": "'el",
    "Elas": "'elaS",
    "cloud": "'eng",
    "type of animal": "'er",
    "fore": "'et",
    "sword": "'etlh",
    "that (previous topic)": "'e'",
    "be beautiful, handsome": "'IH",
    "listen": "'Ij",
    "be sincere": "'Il",
    "oath": "'Ip",
    "vow, swear": "'Ip",
    "be sad": "'IQ",
    "calendar": "'ISjaH",
    "be depressed": "'It",
    "be advanced, highly developed": "'Itlh",
    "altitude": "'Iv",
    "who?": "'Iv",
    "blood": "'Iw",
    "tunnel": "'och",
    "invent, devise": "'ogh",
    "it": "'oH",
    "be thirsty": "'oj",
    "verify": "'ol",
    "be cunning, sly": "'ong",
    "Organia": "'orghen",
    "Organian Peace Treaty": "'orghen rojmab",
    "Organian": "'orghengan",
    "represent": "'oS",
    "emissary": "'oSwI'",
    "compete": "'ov",
    "ache, hurt, be sore": "'oy'",
    "ache, pain, sore": "'oy'",
    "aft": "'o'",
    "be heavy": "'ugh",
    "have a hangover, be hung over": "'uH",
    "be qualified": "'um",
    "dinner": "'uQ",
    "treason": "'urmang",
    "leg": "'uS",
    "be essential, necessary": "'ut",
    "officer": "'utlh",
    "press down": "'uy",
    "million": "'uy'",
    "universe": "'u'",
    "Yes.": "HIja'",
    "No.": "ghobe'",
}

klingon_eval = {
    "I've done it! I've finished!": "pItlh",
    "Well! Aha!": "toH",
    "How did this happen?": "chay'",
    "What's going on?": "jIyajbe'",
    "I don't understand.": "jIyajbe'",
    "I don't care.": "jISaHbe'",
    "No problem!": "qay'be'",
    "Do you speak Klingon?": "tlhIngan Hol Dajatlh'a'",
    "I cannot speak Klingon.": "tlhIngan Hol vIjatlhlaHbe'",
    "Where is a good restaurant?": "nuqDaq 'oH Qe' QaQ'e'",
    "Where is the bathroom?": "nuqDaq 'oH puchpa''e'",
    "How much fuel do we have left?": "nIn 'ar wIghaj",
    "I won't (do it)!": "Qo'",
    "Feed him!": "yIje'",
    "You are right.": "bIlugh",
    "You are wrong.": "bIlughbe'",
    "Am I disturbing you?": "qaSuj'a'",
    "It's not my fault.": "pIch vIghajbe'",
    "My chronometer has stopped.": "tlhaqwIj chu'Ha'lu'pu'",
    "The engine is overheating.": "tujqu'choH QuQ",
    "Where can I get my shoes cleaned?": "nuqDaq waqwIj vIlamHa'choHmoH",
    "Will it hurt?": "'oy''a'",
    "Beam me aboard.": "HIjol",
    "Activate the transport beam!": "jol yIchu'",
    "Surrender or die!": "bIjeghbe'chugh vaj bIHegh",
    "We will meet in the cocktail lounge.": "tachDaq maghom",
    "Your nose is shiny.": "boch ghIchraj",
    "Always trust your instincts.": "Duj tIvoqtaH",
    "There are Klingons around here.": "naDev tlhInganpu' tu'lu'",
    "Don't tell him/her!": "yIja'Qo'",
    "Come here!": "HIghoS",
    "Go to jail.": "bIghHa'Daq yIghoS",
    "Put him on screen.": "yIHotlh",
    "That is unfortunate.": "Do'Ha'",
    "Understood. I understand.": "jIyaj",
    "Success!": "Qapla'",
    "You will be remembered with honor.": "batlh Daqawlu'taH",
    "Animal!": "Ha'DIbaH",
    "There's nothing happening here.": "naDev qaS wanI' ramqu'",
    "Is that understood?": "yaj'a'",
    "Your ship is a garbage scow.": "veQDuj 'oH DujlIj'e'",
    "I have a headache.": "jIwuQ",
    "Hurry up!": "tugh",
    "Very good! Well done!": "majQa'",
    "What do you want?": "nuqneH",
    "Okay.": "luq",
    "When will the water be hot?": "ghorgh tujchoHpu' bIQ",
    "Is this seat taken?": "quSDaq ba'lu''a'",
    "I can't find my communicator.": "QumwI'wIj vItu'laHbe'",
    "This helmet suits you.": "Du'IHchoHmoH mIvvam",
    "You need a rest.": "bIleSnIS",
    "Pay now!": "DaH yIDIl",
    "Four thousand throats may be cut in one night by a running man.": "qaStaHvIS wa' ram loSSaD Hugh SIjlaH qetbogh loD",
    "Revenge is a dish which is best served cold.": "bortaS bIr jablu'DI' reH QaQqu' nay'",
    "How much do you want for that?": "Dochvetlh DIlmeH Huch 'ar DaneH",
    "I'm lost.": "jIHtaHbogh naDev vISovbe'",
    "I can't eat that.": "Dochvetlh vISoplaHbe'",
    "I can't drink that.": "Dochvetlh vItlhutlhlaHbe'",
    "Go away!": "naDevvo' yIghoS",
    "What do I do with this? (i.e., How do I use this?)": "chay' Dochvam vIlo'",
    "What do I do with this? (i.e., Where do I put this?)": "nuqDaq Dochvam vIlan",
    "I've never seen him/her before.": "vIleghpu'be' not",
    "I didn't do it.": "vIta'pu'be'",
    "I wasn't there.": "pa' jIHpu'be'",
    "You seem unhealthy.": "bIpIvHa'law'",
    "You're very ugly.": "bImoHqu'",
    "You lie.": "bInep",
    "Become quiet!": "yItamchoH",
    "Don't speak!": "yIjatlhQo'",
    "Stop speaking!": "bIjatlh 'e' yImev",
    "Where do I sleep?": "nuqDaq jIQong",
    "Does it bite?": "chop'a'",
    "Will you read my manuscript?": "ghItlh vIghItlhta'bogh DalaD'a'",
    "Where do you keep the chocolate?": "nuqDaq yuch Dapol"
}


In [ ]:
#@title **Elvish(Sindarin) Train-Test Data**

elvish_phrases = {
    "abandon": "awartha-",
    "abandonment": "awarth",
    "abhor": "fuia-",
    "abhorrence, horror, fear, dread, detestation, loathing": "delos",
    "abhorrent, detestable, abominable": "thoren",
    "abhorrent, detestable, abomionable, foul": "thaur",
    "abiding, steadfast": "him",
    "abiding, true": "thand",
    "able-bodied men (troop of), manhood, manpower, host, regiment, people, region, wilderness": "gwaith",
    "abound": "ovra-",
    "about, around": "os-",
    "about, concerning": "o",
    "above": "or-",
    "abundant": "ovor",
    "abyss, chasm, void, gulf": "dath, iâ",
    "accordong to, as, like": "be",
    "according to": "ben",
    "account, tale": "trenarn",
    "acre, sown field": "rîdh",
    "across, beyond": "athar",
    "across, over, beyond": "thar-",
    "action, readiness for, vigour, fiery spirit": "hûr",
    "acute, keen, sharp": "laeg",
    "administrative district, shire": "trann",
    "admiration (exclamation of)": "elo!",
    "adult male, male": "dîr",
    "adviser": "saelor",
    "affect, concern": "pessa",
    "affect, trouble, disturb": "presta-",
    "affection or umlaut of vowel": "prestanneth",
    "affection, love, kindness": "mîl",
    "affliction, heavy burden": "caul",
    "after": "ab (ab-)",
    "again, against": "ad",
    "again, back, second": "ad-",
    "against": "dan",
    "age, cycle": "andrann",
    "aged, long-lived": "iphant",
    "agile, swift": "celeg",
    "ago": "ia, io",
    "Ah!": "alae!",
    "air (as a region)": "gwelwen, gwilith",
    "air (as substance)": "gwelu",
    "air (of music), song, tune, singer, also used of rivers": "lind",
    "alas": "nae",
    "alive": "cuin",
    "all": "pân",
    "alone, first, single": "eriol, erui",
    "alone, lone, one": "er-",
    "alone, single, first": "er",
    "alphabet": "tiwdi",
    "alphabet (runic)": "certhas",
    "always, everlasting, eternal": "ui, ui-",
    "amputate, cut round": "osgar-",
    "anchorage, haven, harbour, quiet water": "lorn",
    "ancient, old, former": "iaur",
    "and": "a, ah, ar",
    "anger": "ruith",
    "angle": "bennas",
    "angle, sharp end, point, corner": "nass",
    "animal (quadrupedal mammal)": "lavan",
    "annals": "ínias",
    "anniversary day": "edinor",
    "answer, reply giving new information": "dangweth",
    "anybody, one, somebody": "pen",
    "apostrophe": "gasdil",
    "apparition (spectral or vague)": "auth",
    "appear, seem": "thia-",
    "apple (small red)": "cordof",
    "approach": "anglenna-",
    "April": "gwirith",
    "arch, bow, crescent": "cû",
    "arch, crescent, bow": "cû",
    "area (limited or artificially defined)": "sâd",
    "arise, rise": "eria-",
    "arm": "ranc",
    "army, host": "gweth",
    "arrange, compose": "partha-",
    "arrow": "pilin",
    "art, craft, handcraft": "maenas",
    "article for exchange, ware, thing": "bach",
    "articulation, link": "lîf",
    "as, accordong to, like": "be",
    "as, like": "sui",
    "ash, dust, sand": "ast, lith",
    "ashen, ashy": "lithui",
    "assembly": "hûd",
    "aswer, response": "dambeth",
    "at, with, by near": "na",
    "August, hot": "úrui",
    "autumn": "dannas, iavas, lasbelin",
    "awakening": "echui",
    "away, forth, out": "e-, ed-",
    "away, remote": "gwachae",
    "awe": "anwar",
    "axe, axe blade": "hathol",
    "axe-stroke": "hast",
    "babble": "glavra-",
    "babbling": "glavrol",
    "babe, also used in children's play as the name of the little finger": "laes",
    "baby, little finger": "gwinig",
    "back, again, second": "ad-",
    "bad, evil": "ulug, um",
    "bad, poor, mean": "faeg",
    "bald": "rûdh",
    "ball, globe, mound": "coron",
    "ban, prohibit": "boda-",
    "bane": "dagnir",
    "bank, river-bank": "raw",
    "bank, shore": "esgar",
    "bark, peel": "rîf",
    "barrow": "gorthad",
    "barrow, burial mound, grave, tomb": "haudh",
    "battle": "dagor",
    "battle of two or a few": "maeth",
    "battle, war": "auth",
    "bay": "côf",
    "be able to": "pol-",
    "be able to, can": "pul-",
    "be able, hold, have, possess, can": "gar-",
    "be alive": "cuina-",
    "be off!": "ego!",
    "beach, shore": "faur",
    "beach, shore, coast, strand, line of surf, foaming shore": "falas",
    "beacon, fire-sign": "narthan",
    "bear": "brôg, graw, medli",
    "bear, carry": "col-",
    "beard": "fang",
    "bearer": "cyll",
    "bearish, of bear": "medlin",
    "beat": "dringa-",
    "beat (wings)": "blab-",
    "beaten track, pathway": "bâd",
    "beautiful, fair": "bain",
    "beauty": "baneth",
    "bed": "haust",
    "bedridden, sick": "caeleb",
    "beech": "neldor",
    "beech, mast": "fêr",
    "beget": "edonna-, onna-",
    "begin suddently and vigorously, have an impulse": "heria-",
    "begun again, new": "eden",
    "behind": "adel",
    "bell": "nell",
    "beneath, in, under": "dî",
    "bent, bowed, bowshaped": "cûn",
    "beryl, Elf-stone": "edhelharn",
    "betray, cheat": "gweria-",
    "betrayer": "gwarth",
    "between": "im-",
    "between, within": "im, im-",
    "beyond": "athan",
    "beyond, across": "athar",
    "beyond, across, over": "thar-",
    "bind": "gwedh-",
    "bind, tie": "nod-",
    "birch tree": "brethil",
    "bird (small)": "aew, fileg",
    "birthday": "nostor",
    "bite": "nag-",
    "bite, gnashing ot teeth in grief, woe": "naeth",
    "bitter": "saer",
    "bitterly cold": "helch",
    "black metal": "galvorn",
    "black, dark": "môr, morn",
    "black, swart, swarty, shady, shadowy": "donn",
    "blade (of a broadsword or axe": "hathel",
    "blazing fire": "ruin",
    "blessing, good fortune": "galu",
    "blocked, stopped, closed": "tafnen",
    "blood": "iâr, iûr, sereg",
    "bloodstain": "agarwaen",
    "blossom": "edlothia-, gwaloth",
    "blossom, flower": "loth",
    "blow with fist, clenched fist": "drambor",
    "blow, heavy stroke": "dram",
    "blue": "luin",
    "board, plank": "pân",
    "boat": "lunt",
    "body of Orcs, din, uporoar, tumult, shouting, confused noise": "glam",
    "body, flesh": "rhaw",
    "bogey, monster, orc": "urug",
    "bold": "beren, cand",
    "bolster, pillow": "nedhu",
    "bond of frienship": "gwend",
    "bond of loyalty, bond": "gwedh",
    "bond, bond of loyalty": "gwedh",
    "bond, troth, oath": "gwaedh",
    "book": "parf",
    "border": "edrain, glân, rain, rîw",
    "born": "onnen",
    "born after": "Abonnen",
    "boss, round knob": "dolt",
    "both, two, twi-": "ui-",
    "bound": "naud",
    "bound, limit, enclose": "glenia-",
    "boundary": "gland",
    "boundary marker": "glandagol",
    "boundary, boundary line": "taeg",
    "boundary, lest, girdle": "lest",
    "bounded or defined place": "garth",
    "bow": "peng",
    "bow, arch, crescent": "cũ",
    "bowed, bent, bowshaped": "cûn",
    "bowstring": "tang",
    "braided tress of hair": "finnel",
    "branch": "golf, olf",
    "brand": "ylf",
    "bread": "bass",
    "bread giver (fem.)": "bessain",
    "bread-giver (fem.)": "bassoneth",
    "break out suddenly": "breitha-",
    "breaker, foam": "falf",
    "breath": "thûl",
    "breathe": "thuia-",
    "breeze, puff": "hwest",
    "bride": "dîs, dineth",
    "bride, lady": "dî",
    "bridegroom": "daer",
    "bridge": "iant, ianu",
    "bright spark, star": "gîl",
    "brillance, gleaming": "fael",
    "brilliance, glitter, glory": "aglar",
    "brilliance, glittering light, flame": "rill",
    "brilliance, glory, glitter": "agar",
    "brilliance, light (bright), sunlight, radiance, glittering reflection": "galad",
    "brilliance, radiance": "glaw",
    "brilliant": "celair",
    "brim, brink": "rhîf",
    "bring, lead": "tog-",
    "brink, sharp edge, sudden end": "lanc",
    "brooch, pin": "tachol",
    "broth, liquid food, soup": "salph",
    "brother": "hanar, tôr",
    "brother relative": "gwanûr",
    "brown": "rhosg, rosc",
    "brown, russet, red": "rhosc",
    "brown, swart, dark brown, yellow brown, golden-brown": "baran",
    "bud, sprout": "tui, tuiw",
    "build, make, do": "car-",
    "builder, carpenter, wright": "thavron",
    "building, deed, house": "cardh",
    "building, house": "adab, car",
    "bull": "mund",
    "burial mound, barrow, grave, tomb": "haudh",
    "burn": "dosta-",
    "burning, fiery red": "ruin",
    "bush (low-growing)": "toss",
    "bush, plant": "aiglos",
    "but, however": "dan",
    "butterfly": "gwilwileth, gwiwileth",
    "cairn, pile of stones": "sarnas",
    "cake": "cram",
    "call out": "ialla-",
    "call, name": "esta-",
    "call, shout, cry out": "can-",
    "calligrapher": "tegilbor",
    "can, be able to": "pul-",
    "can, be able, hold, have, possess": "gar-",
    "canopy": "daedelu, orthelian",
    "cape": "ras, rast",
    "cape, headland": "cast",
    "captain": "hest",
    "carpenter, builder, wright": "thavron",
    "cat": "muig, múr, myril",
    "catch": "gad-",
    "catch in a net": "raeda-",
    "causeway": "gondrath, gathrod",
    "cave": "fela",
    "cave, delving, large excavation": "groth, roth",
    "cave, excavation, underground dwelling": "grôd",
    "cave, vault, vaulted ceiling, hall with vaulted roof": "rond",
    "cavern": "gath",
    "cavernous": "raudh",
    "center, core, middle": "enedh",
    "center, heart, core": "nest",
    "cessation, pause, halt, respite": "post",
    "chain, iron bond": "angwedh",
    "chair": "ham",
    "chamber": "sam",
    "channel (water), watercourse, stream, lode, vein": "rant",
    "chant, sing": "linna-",
    "charm, enchant": "lútha-",
    "charm, enchantment, magic": "lûth",
    "chaser (hound of chase)": "rhîn",
    "chasm, abyss, void, gulf": "dath, iâ",
    "cheat, betray": "gweria-",
    "chief, main, prime, prominent": "main",
    "child": "hên",
    "choking, gasping, thirsty": "faug",
    "choose": "cil-",
    "circle": "corn, rind, ringorn",
    "circle, outer ring": "echor",
    "circular": "rend",
    "circular, round": "corn",
    "circular, round, globed": "corn",
    "city with watchtower": "minas",
    "city, town": "canas, caras",
    "city, town with wall around": "ost",
    "clamour, outcry, cry, shout": "caun",
    "clan, family, house": "noss, nost",
    "clasp, holder, socket, hasp, staple": "taew",
    "claw, hook, crook": "gamp",
    "clean, tidy, neat": "puig",
    "clear, make clear": "glanna-",
    "clear, sparkling": "lim",
    "clear, sparkling, light": "hâl",
    "clear, white": "glân",
    "cleared, plain, flat, wide, open": "laden",
    "clearing in forest": "lant",
    "cleaver, sword": "crist, rist",
    "clef, ravine": "riss",
    "cleft": "rist",
    "cleft, cutting, pass": "cirith",
    "cleft, forked, split": "thanc",
    "cleft, gulf, ravine": "criss",
    "cleft, pass between hills, gorge": "cîl",
    "cleft, ravine": "falch",
    "cleft, ravine, cut": "rest",
    "clenched fist, blow with fist": "drambor",
    "clever, skilled": "maen",
    "climb, course, riverbed": "rath",
    "climb, high pass": "andrath",
    "cloak": "coll",
    "close": "hol-, sol-",
    "closed": "hollen, sollen, uidafnen",
    "clothe": "hab-, hamma-",
    "clothes": "hab",
    "clothing": "hammad",
    "cloud": "faun",
    "cloud, veil, manifested body of a Vala": "fân",
    "cloudy": "fanui",
    "club": "grond",
    "co-, como-, together": "gwa-",
    "coin (name of a coin)": "mirian",
    "cold": "ring",
    "cold pool, lake": "hoth, rim",
    "collection of leaves, foliage": "golas",
    "come": "tol-",
    "commanding": "conui",
    "community": "govannas",
    "compact, contract, treaty": "gowest",
    "complete, full, whole": "pant",
    "compulsion, duress, need, oppression, tyranny": "thang",
    "conceal": "delia-, doltha-",
    "conceive": "nautha-",
    "confirm, make firm, establish": "tangada-",
    "conquer, master": "orthor-",
    "consel, warn": "gor-",
    "constrain, oppress": "baugla-",
    "constrainer, oppressor, tyrant": "bauglir, baugron",
    "continually": "him",
    "contorted, twisted, knotted, crabbed, hard": "norn",
    "contrivance, device, machine": "gaud",
    "control, mastery, power, master, victor, lord": "tûr",
    "control, wield": "tortha-",
    "cooked food": "aes",
    "cool": "him",
    "copper": "rust",
    "copper-coloured": "ross",
    "cord": "nordh",
    "corn (not maize)": "iaw",
    "corpse": "daen",
    "corrupt, rotten": "thaw",
    "count, number, reckon": "nedia-",
    "count, reckoning": "genediad",
    "count, sum up": "gonod-",
    "countless, innumerable": "aronoded",
    "couplet": "linnod",
    "course": "ýr",
    "courtyard": "pand",
    "cover, veil, screen": "escal",
    "craft, art, handcraft": "maenas",
    "creature": "ûn",
    "creature powerful, hostile and terrible": "graug",
    "crescent moon": "cúron",
    "crest of a helmet": "amloth",
    "crooked": "raen",
    "cross": "athra-, trevad-",
    "cross, traverse": "athrada-",
    "crossing, ford": "athrad",
    "crossway": "tharbad",
    "crow": "corch, craban",
    "crowd, great number, host": "rim",
    "crowd, host, horde": "hoth",
    "crowd, multitude": "ovras",
    "crown, wreath, garland": "rî",
    "crowned": "rîn",
    "cruel": "balch",
    "cruel, oppressive, tyrannous": "baug",
    "cry (out)": "nalla-",
    "cry of encouragement in battle": "hûl",
    "cry, shout": "ial",
    "crystal": "ivor",
    "cunning (device)": "curu",
    "cunning, wily": "coru",
    "curse": "rhach",
    "custody, prison": "band",
    "custom, habit": "haew",
    "cutlass, sword": "lang",
    "cut, tear": "rista-",
    "cycle, age": "andrann",
    "daddy": "ada",
    "dagger, knife": "sigil",
    "daily": "ilaurui",
    "daisy": "eirien",
    "damp, wet, tearful": "nîd",
    "dance": "liltha-",
    "dangle, hang": "glinga-",
    "dare": "bertha-",
    "dark": "dûr",
    "dark sorcery": "guldur",
    "dark, dusky, misty, obscure": "doll",
    "darkness": "môr",
    "darkness, gloom, night, dead of night": "fuin",
    "darkness, night, nightfall, dusk": "dû",
    "daughter, girl, maid": "iell, sell",
    "dauntless, steadfast, hero": "thalion",
    "dawn, morrowdim, twilight": "minuial",
    "day": "arad",
    "day, daylight": "calan",
    "day, morning": "aur",
    "dazzling white, white as snow": "gloss",
    "dead": "fern, gorth",
    "dead person": "gorth",
    "dead, departed": "gwann",
    "deadly fear": "gorgor",
    "deadly, hateful, fell": "delu",
    "Deads (group of)": "gorthrim",
    "deal with, handle, wield, manage": "maetha-",
    "dear": "mail, mell, muin",
    "dearh": "gurth",
    "death": "gûr, guruth",
    "death (act of dying)": "gwanath",
    "death by fading": "gwanu",
    "death-horror, dread of death": "guruthos",
    "debate": "athrabeth",
    "December": "Girithon",
    "declivity, slope": "pend, pind",
    "deep lore, magic": "angol",
    "deep val, dell": "im",
    "deep valley": "tûm",
    "deep valley, glen": "imlad",
    "deep, low, low-lying": "nûr, tofn",
    "deer": "aras",
    "defend, keep": "gartha-",
    "defile, pass between high walls": "aglonn",
    "defined or bounded place": "gardh",
    "deformed, hideous creature, monster": "ulunn",
    "delight": "gella-",
    "delved mine": "sabar",
    "demon, terrible creature": "raug",
    "denial": "ubed",
    "depart, die": "gwanna-",
    "departed": "gwanwen",
    "departed spirit": "mân",
    "deprive, wrong": "neitha-",
    "descendant, son": "iôn",
    "descendant, son (coll. pl)": "ionnath",
    "desease, sickness": "lhîw",
    "desire": "aníra-, íra-",
    "detestable, abhorrent, abominable": "thoren",
    "detestable, abhorrent, abomionable, foul": "thaur",
    "device, contrivance, machine": "gaud",
    "dew": "midh",
    "dialect, tongue, language": "lam",
    "die, depart": "gwanna-",
    "die, fade": "fir-",
    "difficulty, stiffness, toughness": "tarias",
    "dim light, shade, shadow": "gwâth",
    "dim, overshadow, veil, obscure": "gwathra-",
    "dim, shadowy": "gwathren",
    "din, uporoar, tumult, shouting, confused noise, a body of Orcs": "glam",
    "dire, terrible, fell": "goeol",
    "dirty, soiled": "gwaur",
    "disgust": "dêl",
    "distance": "haered",
    "distant, far": "haeron",
    "distant, far, on the other side, further": "hae",
    "distant, far, remote": "haer",
    "distinct, unique, single": "minai",
    "divine": "bâl",
    "divine power, divinity": "bâl",
    "divinity, Vala": "balan",
    "do battle, make war": "dagra-",
    "do, build, make": "car-",
    "doer, maker": "ceredir",
    "dog": "hû",
    "dog (hunt)": "huan",
    "doing, making": "cared",
    "dome": "telu",
    "don't": "avo!",
    "don't do": "av-",
    "doom": "bartha-",
    "doom, fate": "amarth",
    "doomed": "barad",
    "door": "and",
    "door, gateway": "fennas",
    "door, threshold": "fend",
    "Doriath people": "iathrim",
    "dot": "peg",
    "double": "edaid, tadol",
    "doubt, worry": "trass",
    "dough": "maeas",
    "dove": "cugu",
    "down": "dad, torn",
    "dragon, snake, serpent": "lhûg",
    "drain": "sautha-",
    "dread, fear": "achas, gae",
    "dread, great fear": "goe",
    "dread, horror, fear": "gôr",
    "dread, terror": "gost",
    "dreadful, fearful, holy": "gaer",
    "dreadful, sad, woeful": "naer",
    "dream": "ôl, oltha-",
    "dreariness, twilight, shadow, vagueness": "muil",
    "drink": "soga",
    "drinking vessel": "ylf",
    "driven under compusion": "horn",
    "drought": "suith",
    "drunk": "sogannen",
    "dry": "parch",
    "dungeon, prison": "gador",
    "dusk": "moth",
    "dusk, twilight": "tinnu",
    "dust, ash, sand": "lith",
    "dust, sand, ash": "ast",
    "Dwarf": "hadhod, naug, naugol, nawag, nogoth, norn",
    "Dwarf, Master of Stone": "Gonhir",
    "dwarfed, stunted": "naug",
    "dwarflet": "nogotheg",
    "Dwarfs (people)": "Hadhodrim, naugrim, nogothrim",
    "dwell": "dortha-",
    "dwelling place, land, region": "dôr",
    "dwelling, house, home, family, land, earth": "bâr",
    "eager, fiery": "bara",
    "eagle": "thôr, thoron",
    "ear": "lhewig",
    "earth": "cae, ceven",
    "earthen": "cefn",
    "earthenware": "cefnas",
    "East": "rhûn",
    "East, Orient, sunrise": "amrún",
    "Easterlings, Elves of Beleriand": "Morben",
    "Eastern": "rhúnen",
    "eat": "mad-",
    "eavesdrop, listen in": "lathra-, lathrada-",
    "eavesdropper": "lathron",
    "eavesdropper (fem)": "lethril",
    "ebb": "dannen",
    "echo": "glamor, lamath",
    "echoing": "glamren",
    "eddy": "hwind",
    "eight": "toloth",
    "eighth": "tollui, tolothen",
    "Elda": "calben",
    "elder": "einior",
    "eldest": "iarwain",
    "elephant": "annabon",
    "eleven": "minib",
    "Elf": "edhel, elen",
    "Elf of Aman": "gwanwel",
    "Elf of Beleriand, also used of Easterlings, Avari": "avar",
    "Elf of Beleriand, West-Elf, including Noldor and Sindar": "dúnedhel",
    "Elf-friend": "elvellon",
    "elf-man": "ellon",
    "Elf-stone, beryl": "edhelharn",
    "elf-woman": "elleth",
    "elm tree": "lalf, lalorn, lalven, lalwen",
    "Elve of Aman": "gwanwen",
    "Elves (people)": "Eledhrim, Elrim",
    "Elves of Lórien, people of the Trees": "galadhrim",
    "elvish": "edhellen",
    "embers": "iûl",
    "eminent, lofty, superior": "orchal",
    "empty": "cofn",
    "empty, void": "lost",
    "encampment": "echad, estolad",
    "enchantress": "luithien",
    "encircling fence, outer fence": "ephel",
    "enclose, bound, limit": "glenia-",
    "enclosed": "thaur",
    "enclosed dwelling, walled house or village": "gobel",
    "enclosed grassland, field": "parth",
    "enclosure, garth": "pêl",
    "enclosure, mound": "cerin",
    "end": "meth, methed, methen",
    "end, hindmost part, rear": "tele",
    "endurance": "bronwe",
    "endure": "brenia-",
    "endure, stay, wait, remain, last": "dartha-",
    "enduring": "bronadui",
    "enemy": "goth",
    "enemy, enmity": "coth",
    "enemy, foe": "gûd",
    "enfold": "gwaeda-",
    "enlaced, netted": "raen",
    "enlarge, open": "panna-",
    "enmesh, entangle": "gonathra-",
    "enough": "far, farn",
    "Ent": "onod",
    "entanglement": "gonathras",
    "enter": "minna-, neledhia-",
    "enter, go in": "neledh-",
    "Ents (people)": "onodrim",
    "erratic, wandering": "rain",
    "error": "mist",
    "especially": "edregol",
    "establish, confirm, make firm": "tangada-",
    "estuary, mouth or outflow of a river": "ethir",
    "eternal, always, everlasting": "ui, ui-",
    "eternal, everlasting": "uireb",
    "eternity": "uir",
    "evening, twilight": "aduial",
    "evil creature": "uruk",
    "evil knowledge, magic, sorcery, necromancy": "gûl",
    "evil, wicked": "ogol",
    "exact": "taer",
    "exalted, high": "hall",
    "exalted, lofty, noble": "arth",
    "excavate, hollow out": "rosta-",
    "excavation, hole, lair": "torech",
    "exceedingly, very": "dae",
    "excluded, separate, not common, private": "said",
    "Exiled": "Edledhron",
    "exiled": "edlenn, Edledhron",
    "expression, look, face": "thîr",
    "extended point (wing, horn)": "rafn",
    "eye": "hend",
    "eyed, having eyes": "heneb",
    "face, front": "nif, nîf",
    "fade, die": "fir-",
    "fade, wither": "pel-",
    "fading, withering": "peleth, pelin",
    "faintness, giddiness": "hwîn",
    "fair-minded, just, generous": "fael",
    "faith, hope, trust": "estel",
    "faithful one": "sador, sadron",
    "faithful, strong, loyal": "thenid, thenin",
    "fall": "danna-, lant",
    "fall, falling": "dant",
    "fallen": "dannen",
    "falling, slipping, insecure": "talt",
    "fallow, pale": "malu",
    "false, hidden, twicked": "raeg",
    "family": "nothrim",
    "fane, holy place, sanctuary": "iaun",
    "fang": "carch",
    "fantastic": "hwiniol",
    "far off": "palan-",
    "far, distant": "haeron",
    "farewell": "navaer, novaer",
    "farthing, fourth part": "canath",
    "fashion, make, shape": "echad-",
    "fasten, tie": "taetha-",
    "fat, thick": "tûg",
    "fate, final end, fortune": "manadh",
    "fated": "amarthan",
    "father": "adar",
    "Father of Men": "adanadar",
    "fathom": "raew",
    "fawn, pale": "maidh",
    "fear": "groga-",
    "fear, feel terror": "gosta-",
    "fear, pallor": "niphred",
    "feast, festival": "mereth",
    "feather": "pess",
    "February": "nínui",
    "feel, stroke, handle, wield": "matha-",
    "fell with the hand": "plada-",
    "female": "inu",
    "fence": "iâth, thora-",
    "fence with spikes and sharp stakes": "cail",
    "fenced, guarded, hidden": "thoren",
    "ferm, robust": "tarch",
    "festive, joyous": "meren",
    "fetch, make come": "toltha-",
    "few": "sun",
    "field, enclosed grassland": "parth",
    "field, garden, yard": "sant",
    "fierce, wild": "braig",
    "fierceness": "bregolas",
    "fifth": "lefnui",
    "filament, spider filament": "lhê",
    "fill": "panna-, pathra-",
    "find": "hir-, tov-",
    "find a way, make a way": "rad-, rada-",
    "fine pierced hole": "tess",
    "fine, high, lofty, noble": "brand",
    "fine, slender": "lhind",
    "finger": "lebed, leber",
    "fire demon": "balrog",
    "fire, flame, sun": "naur",
    "fire, heat": "ûr",
    "fire-dragon": "urlug",
    "firm": "tanc",
    "firm, steady, stalwart": "thala",
    "first": "minui",
    "fish": "gail, hâl, lim",
    "fish-dragon, sea serpent": "limlug",
    "fist": "dond, paur",
    "fit, useful, good": "maer",
    "five": "leben",
    "fix, set": "penia-",
    "flame": "lacha-, than-",
    "flame-eyed, Elf who has lived in the Blessed Realm": "lachenn",
    "flat": "talu",
    "flat field, low": "talf",
    "flatlands, flat surface, plane, plain": "talath",
    "flee": "dreg-, drega-",
    "flee!": "drego!",
    "fling, fly, rush": "rib-",
    "float": "loda-",
    "flood-water, wash": "iôl",
    "floor": "panas, talaf, talan",
    "flow": "siria-",
    "flow like a torrent": "rimma-",
    "flower of gold": "mallos",
    "flowering": "edlothiad",
    "flowering valley": "imloth",
    "flowing": "cell",
    "fly, sail, wander, stray": "renia-",
    "fly, wander, sail": "revia-",
    "flying, rushing": "rimp",
    "foam": "faltha-, gwing",
    "foe, enemy": "gûd",
    "fog, mist": "híth, hithu",
    "foggy, misty": "hithui",
    "foggy, obscure, vague": "hethu",
    "follow": "aphada-",
    "follower": "aphadon, echil",
    "follower, vassal": "býr",
    "followers": "aphadrim",
    "food": "mann",
    "foot": "tâl",
    "foot (of an animal)": "pôd",
    "footprint, track": "rein",
    "footstool, hassock": "tharas",
    "for": "an-, an",
    "for a long time, long": "anann",
    "for me": "anim",
    "forest": "taur",
    "Forest people (Silvan Elves)": "Tawarwaith",
    "forest, wood": "eryn, tawar",
    "forester": "tauron",
    "forest-fire": "erynaur",
    "forge": "tamin",
    "forgive": "dihena-, gohena-",
    "forked, cleft, split": "thanc",
    "formed, shaped": "cadu",
    "forsaken": "eglan",
    "Forsaken (people), Sindar": "Eglath",
    "Forsaken, Sindar, Falathrim": "Egladhrim",
    "fort, fortress, tower": "barad",
    "fortress with watchtower": "ostirion",
    "four": "canad",
    "fourth": "cannui, canthui",
    "fox": "rusc",
    "fragile, thin, slender": "nind",
    "frail": "nimp",
    "free, freed": "lain",
    "free, release": "adleitha-",
    "Friday (sky-day)": "ormenel",
    "frien (fem.)": "meldis",
    "friend": "meldir, mellon",
    "friend, lover": "seron",
    "friendly, loving, kind": "milui",
    "frog": "cabor",
    "frost": "niss",
    "frozen": "helegnen",
    "fullness": "pathred",
    "fungus, sponge": "hwand",
    "fur": "heleth",
    "fury": "gruith",
    "gap, opening, mountain pass": "dîn",
    "garden, field, yard": "sant",
    "garment": "hamp",
    "garth, enclosure": "pêl",
    "gate, door": "annon",
    "gaze, guard, watch": "tir-",
    "gaze, look toward, watch, guard": "tiria-",
    "germ, seed": "eredh",
    "giant": "noroth",
    "gift": "ant",
    "girdle, boundary, lest": "lest",
    "girl, virgin, maiden": "-wen",
    "give": "anna-",
    "giver (fem.)": "oneth",
    "giving": "anneth",
    "gladden": "ninglor",
    "glance": "glind, henn",
    "glance at": "glinna-",
    "glance, view, looking": "tîr",
    "glass": "heledh",
    "glint": "tinna-",
    "glint of light": "glîn",
    "glisten": "thilia-",
    "glitter (polish metal)": "ross",
    "gloomy, low": "dofn",
    "gloomy, sad": "dem",
    "glorify, praise": "egleria-",
    "glorious": "aglareb",
    "go": "bad-",
    "go to exile": "edledh-, edledhia-",
    "go up": "dol-",
    "goblet": "sûl",
    "God (the One)": "Eru",
    "going up": "am-",
    "gold (color?)": "mall",
    "gold (metal)": "côl, malad",
    "golden": "mallen, malthen",
    "golden light": "glaur",
    "golden tree": "mallorn",
    "good, fit, useful": "maer",
    "goose": "gwaun",
    "gore, spearhead, promontory": "naith",
    "gorse": "aeglos",
    "grasp, seize": "mab-",
    "grass": "glae",
    "grave": "sarch",
    "gravel": "brith",
    "great fear, dread": "goe",
    "great hall": "thamas",
    "great number": "lae",
    "Great Ocean": "gaearon",
    "great region, province": "ardhon",
    "great sea, ocean": "Aearon",
    "great stone, rock": "gond",
    "great, large": "dae",
    "great, mighty": "beleg",
    "greedy": "melch",
    "green": "calen",
    "Green Elf": "laegel",
    "Green Elves (people)": "laegrim",
    "greet": "suila-, suilanna-",
    "greeting": "suilad",
    "grey": "mithren",
    "Grey-Elf": "send",
    "gross (144)": "host",
    "grow": "gala-",
    "grow toward evening": "thinna-",
    "growth, plant": "galas",
    "guard, guarding": "tirith",
    "guess, idea, notion": "inc",
    "guesthouse": "sennas",
    "guilty, wily, sinister": "crumguru",
    "gull": "gwael, maew, mýl",
    "gulp": "yll",
    "hack through": "hasta-",
    "hail": "suil",
    "hail!": "ai",
    "hair, lock of hair, tress": "fin",
    "hairnet, tressure": "cathrae",
    "half": "per-, perin",
    "Half-Elf": "peredhel",
    "Halfling, Hobbit": "perian",
    "Halfling, Hobbit (People)": "periannath",
    "hall": "tham",
    "halt, stop": "dar-",
    "hammer": "dam, damma-, dring",
    "hand": "cam, mâb",
    "hand's palm": "camlann",
    "hanging": "gling",
    "harborage": "hobas",
    "hard, contorted, twisted, knotted, crabbed": "norn",
    "hare": "laboth",
    "harp": "gannel",
    "harp, play a harp": "ganna-, gannada-",
    "harper": "talagan",
    "hasty, vigorous, impetuous": "gorn",
    "hat": "carab",
    "have knowledge": "ista-",
    "haven": "círbann",
    "he": "hon",
    "head, hill": "dôl",
    "heal": "nesta-",
    "healing": "nestad",
    "heap, mound": "cûm",
    "heart": "hûn",
    "heaven": "menel",
    "heavy": "long",
    "hedge": "cai",
    "hedge of spikes": "caraes",
    "height, summit of high mountain": "taen",
    "heir": "hîl",
    "helm": "thôl",
    "helmet": "harn",
    "herb": "salab, salch",
    "here": "sí",
    "hero": "callon",
    "hew": "drava-",
    "hewn, log": "drafn",
    "hidden, secret": "dolen, thurin",
    "hidden, veiled, shadowed": "hall",
    "hideous, monstrous": "uanui",
    "high, exalted": "hall",
    "hill, steep-sided mount": "amon",
    "hindmost part, rear, end": "tele",
    "historical": "gobenathren",
    "history": "gobennas, pennas",
    "hither, here": "nev-",
    "hold allegiance, serve": "buia-",
    "hole": "gas",
    "hollow": "coll, naw",
    "holly": "ereg, eregdos",
    "holly, thorn": "rêg",
    "holy, dreadful, fearful": "gaer",
    "honey": "gli",
    "honeycomb": "nîdh",
    "hook, claw, crook": "gamp",
    "hop": "laba-",
    "hope": "hartha-, harthad",
    "hope (based on a reason)": "amdir",
    "horizon": "tiriw",
    "horn, peak": "aeg, rasg",
    "horn, trumpet": "rom",
    "horrible": "gortheb",
    "horrible, loathsome": "deleb",
    "horror": "angos, gorog, goroth",
    "horse": "lobor, roch",
    "Horse-Lord, knight, rider": "rochir",
    "Horse. Lords (People)": "Rochirrim",
    "host, army": "gweth",
    "hot": "born",
    "hound of chase": "rýn",
    "household, troop under a lord": "herth",
    "howl": "gawa-",
    "howling": "gawad",
    "human": "firen",
    "hummock": "gwastar",
    "hump": "tump",
    "hungry": "saig",
    "hunt": "rui, fara-",
    "hunt, hunting": "farad, faras",
    "hunter": "faron, feredir",
    "hunters?": "faroth",
    "hurl": "had-",
    "hurler of darts, thrower of darts": "hadron",
    "husband": "hervenn",
    "hymn": "aerlinn",
    "I": "im, ni",
    "ice": "heleg",
    "icicle": "aeglos",
    "idea": "naw",
    "if": "ae",
    "ill deed, sin": "úgarth",
    "ill, sick, sickly": "lhaew",
    "immortal, name of a flower": "alfirin",
    "impetuous, rushing": "alag",
    "impetus": "gorf",
    "in": "mi, vi",
    "in (used of time)": "ned",
    "in, inside": "ne-",
    "incite": "meltha-",
    "incline": "talad",
    "inclined, slant down": "dadbenn",
    "index finger": "lebdas",
    "inner mind, heart": "gûr",
    "insert, stick in": "nestag-",
    "inside": "nedh-",
    "instead, rather": "sennui",
    "insult": "eithad",
    "insult, prick with a sharp point, stab, treat with scorn": "eitha-",
    "intelligent": "hand",
    "intend, mean, resolve": "thel-",
    "intention, purpose": "thelion",
    "iron": "ang",
    "island": "toll",
    "isolated hill": "mindon",
    "isolated, lonely": "ereb",
    "issue of water, spring, well": "eithel",
    "January": "narwain",
    "jaw": "ank",
    "jaw, row of teeth": "anc, carach",
    "jaw, set of jaws": "naew",
    "jerk, twitch, snatch": "ritha-",
    "jerk, twitch, trick": "rinc",
    "jewel, Silmaril": "Mirion",
    "jewel, treasure": "mîr",
    "jewel-like": "míriel",
    "journey, way": "lend",
    "joy": "glass",
    "jubilation": "gellam",
    "judge": "badhor, badhron",
    "judgment, sentence": "baudh",
    "juice": "saw",
    "juicy": "pihen",
    "July": "cerveth",
    "June": "nórui",
    "keen, acute, sharp": "laeg",
    "keep, retain": "heb-",
    "Khuzdûl": "Hadhodren",
    "kindle": "nartha-",
    "kindler (fem.)": "thoniel",
    "King": "âr",
    "king": "aran, taur",
    "kingdom": "arnad",
    "kingdom, realm": "aranarth",
    "kingfisher (bird)": "heledir",
    "kingsfoil": "athelas",
    "kinsman, kinswoman": "gwanur",
    "kiss": "mib-, mítha-",
    "knight, Horse-Lord, rider": "rochir",
    "knock": "tamma-",
    "knot": "nardh",
    "knowledge, lore": "ist",
    "labour, toil": "muda-",
    "lady": "brennil, heryn",
    "Lady": "hiril",
    "lair, resting place": "caew",
    "lake, pool": "ael, lîn",
    "lament": "naergon, noe",
    "lamentation": "nirnaeth",
    "lamp": "calar",
    "lampwright": "calardan",
    "lands of Middle-earth": "ennorath",
    "large": "daer",
    "last": "medui",
    "last day of the year": "penninor",
    "last, survive": "brona-",
    "laugh": "gladha-",
    "laughter": "lalaith",
    "lay, poem song": "glîr",
    "lead, bring": "tog-",
    "leaf": "lass",
    "league, pause, stop": "daur",
    "lean, thin, meager": "lhain",
    "leap": "cab-, cabed",
    "leaping down, swooping": "thôr",
    "learn": "gelia-",
    "learned": "istui",
    "left": "crom, hair",
    "left (direction) south": "hâr",
    "left Behind": "lefn",
    "left hand": "crum, hair",
    "left hand, left side": "harvo",
    "left-handed": "crumui, hargam",
    "lerned in deep arts, wise": "golwen",
    "letter, sign, tengwa": "têw",
    "level, open space": "land",
    "lick": "lav-",
    "lie": "fura-",
    "lie (on the ground": "caeda-",
    "life": "cuil",
    "lift": "hal-",
    "light": "calad",
    "light (bright)": "gail",
    "like silver": "celebren",
    "line, row": "lîr, tê, tî",
    "lion, rush, roaring noise": "raw",
    "listen": "lasta-",
    "little": "pîn, tithen",
    "little brother, also the name of the middle finger": "honeg",
    "little father, thumb": "atheg",
    "little finger": "niged",
    "little kiss": "mib",
    "little river, stream": "cel",
    "live": "cuia-",
    "load": "cûl",
    "loaf, round bread": "basgorn",
    "lofty, eminent, superior": "orchal",
    "log, hewn": "drafn",
    "lonely, isolated": "ereb",
    "long": "and",
    "long and thin": "taen",
    "long mark": "andaith",
    "long year, Valian year": "ennin",
    "Longbeard (tribe of Dwarves": "Anfang",
    "Longbeards (people)": "Anfangrim",
    "look toward, watch, guard, gaze": "tiria-",
    "lord": "brannon",
    "lord, master": "hîr",
    "loud, noisy": "brui",
    "love": "mel-, mela-, meleth, mil-",
    "lover (fem.)": "melethril",
    "lover (masc.)": "melethron",
    "low, flat field": "talf",
    "lust": "mael",
    "lustful": "maelui",
    "lying in bed, sickness": "cael",
    "machine, contrivance, device": "gaud",
    "made of stone, stony": "gondren",
    "mage, wise man": "gollor",
    "magic metal that only mirrors starlight and moolight": "ithildin",
    "maiden": "gwend",
    "make": "tan-",
    "make a way, find a way": "rad-",
    "make come, fetch": "toltha-",
    "maker": "tân?",
    "male": "anu",
    "Man": "adan",
    "Man of the West, Númenórean": "dúnadan",
    "man, husband": "benn",
    "man-spearhead (wedge formation in battle)": "dírnaith",
    "many, fréquent": "laew",
    "March": "gwaeron",
    "mark": "tagol, taith",
    "marking (a boundary)": "taig",
    "mass of ice": "gochel",
    "master": "herdir, heron",
    "May": "lothron",
    "mayor": "condir",
    "me": "nin",
    "meet": "govad-",
    "merry": "gelir",
    "mesh, net": "rem",
    "met": "govannen",
    "metal": "tinc",
    "middle finger": "toleg",
    "middle, center, core": "enedh",
    "Middle-Earth": "Ennor",
    "midyear": "enedhin",
    "mighty, great": "beleg",
    "mind, inner thought, meaning, heart": "ind",
    "mirror": "cenedril",
    "mist, fog": "híth, hithu",
    "mistake, wrong": "athranna-",
    "moisten": "limmida-",
    "Monday (star-day)": "orgilion",
    "monster": "úan",
    "moon": "Ithil",
    "morning, day": "aur",
    "mortal": "fíreb, firin?",
    "Mortal Humains": "firiath",
    "mortal maiden": "fíriel",
    "mortal man": "fair, firion",
    "mortal woman": "adaneth, firieth",
    "mother": "emel, nana, naneth",
    "mother, finger index": "emig",
    "mound": "tund",
    "mountain": "orod",
    "mountain peak": "aegas",
    "mountain stream, torrent": "oll",
    "mountaineer": "orodben",
    "multitude, crowd": "ovras",
    "muscle, vigour, physical strength": "tû",
    "must": "boe",
    "my": "nín",
    "nail": "taes",
    "naked": "hell, lanc",
    "name": "eneth",
    "Nandor": "Danwaith, Denwaith",
    "narrative poem": "glaer",
    "narrator": "pethron",
    "narrow": "agor",
    "neck": "achad, iaeth",
    "neck, throat": "hell",
    "necklace": "sigil",
    "need": "baur",
    "neighbour": "sammar",
    "net": "raef",
    "new": "gwain, sain",
    "new moon": "cýron",
    "news, tidings": "siniath",
    "night, gloom": "maur",
    "nightingale": "dúlinn, merilin, tinúviel",
    "nighttime, gloom": "daw",
    "nine": "neder",
    "ninth": "nedrui",
    "no! don't!": "baw, law",
    "noble": "arod",
    "noble (man)": "arphen",
    "noble lady": "arwen",
    "noble maiden": "rodwen",
    "noble, royal": "arn",
    "Noldo": "gódhel, golodh",
    "Noldo (exiled)": "Odhel",
    "Noldo Jewel (Silmaril)": "Golovir",
    "Noldor (exiiled)": "Ódhellim",
    "Noldor (people)": "Gódhellim, Golodhrim",
    "noose": "nŷw",
    "north": "forod",
    "north (direction)": "forven",
    "north, right": "fôr",
    "northern": "fervain",
    "Northfolk, Northmen": "forodrim",
    "Northmen, Northfolk, north regions": "forodwaith",
    "nose": "nem",
    "not": "al-",
    "not, without": "ú, ú-",
    "novel, tale": "sinnarn",
    "November": "Hithui",
    "now": "si",
    "number, noun": "gwanod",
    "nurse, also used of rivers, air (of music), song, tune": "lind",
    "O!": "a",
    "oak tree": "doron",
    "oath": "gwest",
    "oblique, slanting": "adlant",
    "obstinate": "tarlanc",
    "occasion, time": "lâu",
    "ocean, sea": "aear, gaear",
    "October": "narbeleth",
    "odor": "ûl",
    "of copper": "rustui",
    "of crystal": "ivorcheneb, ivren",
    "of healing": "nestadren",
    "of iron": "angren",
    "of silver": "celevon",
    "of stone": "gonui",
    "of the shore": "falathren",
    "of wool, woolen": "taw",
    "off! be off!": "ego!",
    "Oiolossë": "Uilos",
    "old, worn": "brûn",
    "old, worn (od things)": "gern",
    "on": "bo",
    "on both sides": "ath-",
    "on this side of": "nef",
    "one, number one": "min, mîn",
    "one-handed": "erchamion, erchammon, erchammui",
    "open": "edra-",
    "oppressive, cruel, tyrannous": "baug",
    "or": "egor",
    "orc": "glamog",
    "Orch": "orch",
    "orkish": "erchion",
    "our": "vín",
    "outer ring, circle": "echor",
    "outside, without": "ar-",
    "own, property": "garn",
    "pain": "naeg, naegra-",
    "pair of ears": "lhaw",
    "pair of twin": "gwanûn, gwanûr, gwenyn",
    "palantir, seeing stone": "gwachaedir",
    "pale blue": "elu",
    "pale, glittering": "gael",
    "pale, grey": "mith, thind",
    "pale, white": "nim",
    "palm (hand)": "talf",
    "palm (of hand)": "plâd",
    "parent (fem.)": "odhril",
    "parent (masc.)": "odhron",
    "pasture": "nadhor, nadhras",
    "path": "imrad",
    "path, track": "pâd, râd",
    "paved way": "othlonn",
    "peace": "sîdh",
    "peak": "mîn",
    "pebbly": "brithon",
    "pedlar": "bachor",
    "peeble-bank": "serni",
    "pen": "tegil, tegol",
    "People of Doriath": "Iathrim",
    "People of the South": "haradrim",
    "permission": "dâf",
    "petrel (small gull)": "paen",
    "petty, small, name of the little finger": "niben",
    "Petty-Dwarf": "nogoth niben",
    "pick out, pick up": "leutha-",
    "piercing, sharp, pointed": "megor",
    "pilgrim, wanderer": "randir",
    "pillar (wood)": "thafn",
    "pillow": "pesseg",
    "pimpernel (flower)": "elanor",
    "pine-tree": "thaun, thôn",
    "pinion, wing": "roval",
    "pipe-weed": "galenas",
    "pivot": "pelthaes",
    "play": "teilia-, telia-",
    "play, sport": "teilien, telien",
    "poem, lay, song": "glîr",
    "point of spear": "aith, thela",
    "point, spear point": "ecthel",
    "pointe": "ment",
    "pointed, sharp": "aeg",
    "poison": "saew",
    "pollen, yellow powder": "mâl",
    "pool": "loeg",
    "poplar tree": "tulus",
    "potter": "cennan",
    "precipice": "rhass",
    "prick, point, stick": "ercha-",
    "prick, point, stick, thrust": "nasta-",
    "prickle": "erch",
    "prince": "cund, ernil",
    "princess": "riel",
    "prohibition, refusal": "ablad",
    "protect": "beria-",
    "puddle, small pool": "both",
    "quarrel": "cost",
    "queen": "rîs",
    "quench": "luithia-",
    "race": "nûr",
    "radiance, brilliance": "glaw",
    "radiant": "faen",
    "rain": "eilia-, elia-",
    "rainbow": "eilian, ninniach",
    "rain-bow": "eiliant",
    "rainy": "rost",
    "raise": "ortha-",
    "range of mountain peaks": "aeglir",
    "range of mountains": "orodrim",
    "ransom": "danwedh",
    "rapid, swift": "legrin",
    "rat": "nâr",
    "readiness for, action, vigour, fiery spirit": "hûr",
    "realm, kingdom": "aranarth",
    "realm, region": "ardh",
    "reap": "critha-",
    "recite (poem), sing": "glir-",
    "reckon": "genedia-",
    "recount, tell to end": "trenar-",
    "red": "caran, naru",
    "reed": "lisc",
    "reek, smoke": "osp",
    "refusal, reluctance": "avad",
    "regional, local": "trannail",
    "relative, brother": "gwanûr",
    "remembrance, queen": "rîn",
    "remenber": "ren-",
    "rend": "narcha-",
    "renewal": "cîl",
    "renewed": "cŷr",
    "repose, rest": "îdh",
    "resonant (of strings)": "tong",
    "rest": "hoda-, sedh-",
    "resting place, lair": "caew",
    "reunion": "aderthad",
    "reunite": "adertha-",
    "rider": "rochben, rochon",
    "ridge, spike, stake": "gondrafn, gondram",
    "right hand, north": "fuir",
    "right, north": "fôr",
    "right-handed": "forgam",
    "rigid, sapless, stiff, withered": "tharn",
    "rill, river": "sîr",
    "ring": "corf",
    "ring finger": "lebent, nethig",
    "ringing od bells": "nelladel",
    "ringlet": "laus, loch",
    "river": "celon, sirion",
    "river, large rive with current": "duin",
    "riverbank": "duirro",
    "road, way": "mên",
    "rock, great stone": "gond",
    "rockenhewn hall": "rhûd",
    "roof": "orthel-, toba-, tobas",
    "root": "thond",
    "root (edible)": "solch",
    "rope, strap": "raph",
    "rose": "meril",
    "row, line": "tê, tî",
    "royal": "arnen",
    "ruddy": "gruin",
    "ruddy of face": "crann",
    "run": "nor-",
    "rune": "certh",
    "runic alphabet": "angerthas",
    "rush, fling, fly": "rib-",
    "russet, red, brown": "rhosc",
    "rustling sound, whisper": "lhoss, rhoss",
    "sad": "nûr",
    "saddle": "rogol",
    "sadness, gloom": "dim",
    "sailor, shipman": "cirion",
    "salvation, saving": "edraith",
    "salve": "glaew",
    "Saturday (Vala-day)": "orbelain, Or-Rodyn",
    "say, speak": "ped-",
    "scarlet, red": "coll",
    "scorn, mocking": "iaew",
    "scratch": "rhib-",
    "screen": "haltha-",
    "sea": "aer, gaer",
    "sea weed": "gaeruil",
    "seashell": "half",
    "season of fading (autumn)": "firith",
    "season of stirring (spring)": "echuir",
    "season of summer": "laer",
    "seaweed": "aeruil, uil",
    "second": "edwen, tadui",
    "second in command, supporting": "taid",
    "secret, hidden": "dolen",
    "see": "cen-",
    "seed, germ": "eredh",
    "seeing, sight": "cened",
    "send": "menna-",
    "sensible, wise": "noen",
    "September": "ivanneth",
    "set aside": "seidia-",
    "set of jaws, jaw": "naew",
    "seven": "odog",
    "seventh": "ochui, odothui, othui",
    "sexual desire": "îr",
    "shadow": "lum",
    "shadow with a recongnizable shape": "morchant",
    "shadowy": "gwathui",
    "Shadowy People (of Dunland)": "Gwathuirim",
    "shaggy hair": "fast",
    "shallow lake": "lô",
    "shapely": "cadwar, maed",
    "shaping": "cannas",
    "sharp, penetrating": "maeg",
    "sharp, pointed": "aeg",
    "sharp-eyed": "maecheneb",
    "shaven, smooth (for hills without trees)": "paran",
    "she": "he, hen, hene",
    "shield": "thand",
    "shield wall": "amath",
    "shield wall, shield fence": "thangail",
    "shine white": "síla-",
    "ship": "cair",
    "shipbuilder": "círdan",
    "shipman, sailor": "cirion",
    "shore": "habad",
    "short": "estent, then, thent",
    "shortness": "thinnas",
    "shout, cry": "ial",
    "shudder": "gir-",
    "shuddering, horror": "girith",
    "sick, ill, sickly": "lhaew",
    "sickle": "cerch",
    "sickly": "gem",
    "sickness": "paw",
    "sickness, lying in bed": "cael",
    "sign": "tann",
    "silent": "dínen",
    "silima, the crystal substance of the Silmarils": "silef",
    "Silmaril": "Silevril",
    "silver": "celeb",
    "silver birch (tree)": "nimbrethil",
    "silver spark": "gildin",
    "Sinda (of Doriath)": "iathron",
    "Sinda, One of the Forsaken": "egol",
    "Sindar": "Eluwaith, Mithrim, mithron",
    "sing": "liria-",
    "single flower": "elloth, lotheg, lothod",
    "sinner": "raegdan",
    "sister": "gwathel, muinthel, thêl",
    "sister, girl": "neth",
    "sit": "haf-, hav-",
    "six": "eneg, engui",
    "sixth": "enchui",
    "skilled, handy": "flâd",
    "slain": "dangen",
    "slant down": "penna-",
    "slant, slope": "adlanna-",
    "slave": "mûl",
    "slay": "dag-",
    "sleep": "losta-",
    "slender": "trîw",
    "sling": "hadlath",
    "slim, slender": "fim",
    "sloping, tilted": "adlann, adlod",
    "small bird": "filigod",
    "small finger": "lebig",
    "small forest, wood": "glad",
    "smell": "nosta-, thost",
    "snake": "lýg",
    "snake, serpent": "amlug",
    "snow": "loss",
    "Snow Men (People)": "lossoth",
    "snowdrop": "ninim",
    "snowdrop, flower": "niphredil",
    "snowy": "lossen",
    "soaking wet": "loen",
    "soap": "glûdh",
    "soft": "mae",
    "soil": "cêf",
    "soil, stain": "gwatha-, maw",
    "soldier (Orkish)": "daug",
    "sole of the foot": "telluin",
    "son": "ion",
    "song": "laer",
    "sorcery": "morgul",
    "soul, spirit": "fae",
    "sound bells": "nella-",
    "sound of the bells": "nellad",
    "sound of the horn": "romrú",
    "sound of the trumpets": "rû",
    "source, spring": "celu",
    "south": "harad",
    "southern": "harn",
    "sow": "redh-",
    "spark": "tint, tinu",
    "sparkling-dragon": "faelug",
    "speak, say": "ped-",
    "spear point, point": "ecthel",
    "speed, urge on": "hortha-",
    "spell": "luith",
    "spider": "lhingril, ungol",
    "spike, tooth of rock": "carag",
    "spine": "ech",
    "spirit": "faer",
    "spit": "puia-",
    "splendour, glory": "claur",
    "sponge, fungus": "hwand",
    "spot, slot, track, footprint": "ruin",
    "spring": "ethuil",
    "spring (water)": "celos",
    "spring, swell, sprout": "tuia-",
    "spy": "ethir",
    "stain": "gwass",
    "stain, stained": "mael",
    "stained": "gwaen",
    "stair": "dim",
    "stairway": "pendrath",
    "stake, spike": "ceber",
    "star": "êl",
    "starlight": "gilgalad",
    "Star-Sun (flower)": "elanor",
    "staying": "avorn",
    "steadfast man, trusty man": "bôr",
    "steep": "baradh",
    "steep mountain peak": "tarag",
    "stem": "telch",
    "stench": "thû",
    "stick, prick, point": "ercha-",
    "sticky, viscous": "hîw",
    "stiff grass": "thâr",
    "stiff, tough": "dorn, tara",
    "stink": "thosta-",
    "stirrup": "talraph",
    "stone": "sarn",
    "stonecrop (plant)": "seregon",
    "stop up": "dilia-",
    "stopper, stopping": "dîl",
    "storm of wind": "alagos",
    "story": "pent",
    "straight": "tîr",
    "strap, rope": "raph",
    "straying": "mistad",
    "stream, flowing": "celeth, sirith",
    "street": "othrad",
    "strengh": "bel, bellas",
    "stretch, reach": "raetha-",
    "strip": "heltha-",
    "strong in body": "bell",
    "stunt, stop short, not allow to continue": "nuitha-",
    "successo": "dŷr",
    "sudden": "bragol",
    "sudden, lively, quick": "brêg",
    "sudden, violent": "bregol",
    "suddenly": "breged",
    "suffice": "feiria-",
    "Sun": "Anor",
    "Sunday (sea-day)": "oraearon",
    "sunlight, radiance, gold": "glawar",
    "sunny, fiery": "nórui",
    "sunrise, Orient, East": "amrún",
    "Sunset, West": "annûn",
    "superior, eminent, lofty": "orchal",
    "support": "tulu",
    "suppose": "inga-",
    "surface (flat)": "palath",
    "survival": "bronad",
    "swallow": "tuilinn",
    "swan": "alph",
    "sward": "pathu",
    "sward, turf": "sâdh",
    "swear": "gwesta-",
    "sweet": "melui",
    "sweet, tuneful": "lend",
    "swift": "lagor, lint",
    "sword": "magol, megil",
    "swordsman": "magor",
    "sworn brother": "gwador",
    "table": "talfin",
    "tall": "tond",
    "tangled": "galadhremmen",
    "tea": "lasguil, suithlas",
    "teach": "goltha-, saeda-",
    "tear": "nîn",
    "tear, weeping": "nîr",
    "tearful": "níniel",
    "Teler": "glinnel",
    "Teler (member of the Third Clan of the Elves)": "teler",
    "Teleri (People)": "Telerrim",
    "tell (a story)": "nara-",
    "temptation": "úthaes",
    "ten": "caer, pae",
    "tenth": "paenui",
    "terrify": "gruitha-",
    "thank": "hanna-",
    "thanks": "hannad",
    "that": "i, taw?",
    "thatch": "taus",
    "the": "i",
    "them (inanimates)": "hain",
    "there": "ennas",
    "these": "sin",
    "theth": "paenen",
    "thick, fat": "tûg",
    "thing": "nad",
    "third": "nail, nelui",
    "thirsty, choking, gasping": "faug",
    "thirtieth": "nelchaenen",
    "this": "sen",
    "thong": "lath",
    "thorn": "êg",
    "thoughtfulness": "idhor",
    "thousand": "meneg",
    "thread": "lain",
    "three": "nêl, neled",
    "throne": "archam",
    "through": "tre-, trî",
    "through, together": "godref",
    "thrower": "hador",
    "thumb": "naub",
    "Thusday (tree-day)": "orgaladh",
    "tide": "duinen",
    "tie, bind": "nod-",
    "time, occasion": "lâu",
    "tine, spike, point, sharp horn": "till",
    "tiny": "pigen",
    "tiny, small, frail": "mîw",
    "to be": "na-",
    "today": "sîr",
    "together": "go-",
    "toil, labour": "muda-",
    "tom-cat": "muron",
    "tooth": "nagol, nêl",
    "top": "caw",
    "torment": "baul",
    "torrent": "thórod",
    "tough, stiff": "dorn",
    "tower, fortress, fort": "barad",
    "towering": "mîn",
    "trade": "banga-",
    "trample": "batha-",
    "Trawn Folk (Dwarfs)": "Dornhoth",
    "treaty, compact, contract": "gowest",
    "tree": "galadh, orn",
    "tree (sort of)": "lebethron",
    "tressure, hairnet": "cathrae",
    "triangle": "nelthil",
    "triumph, joy": "gell",
    "triumphant": "gellui",
    "troll": "torog",
    "troth, bond, oath": "gwaedh",
    "trouble": "tass",
    "trouble, harass": "trasta-",
    "trust": "bora-",
    "trusty": "tolog",
    "Tuesday (sun-day)": "oranor",
    "twelve": "ýneg",
    "twelveth": "yngui",
    "twicked, false, hidden": "raeg",
    "twilight": "uial",
    "twin": "gwanunig",
    "twirl, whirl, eddy": "hwinia-",
    "two": "tâd",
    "two-legged": "tad-dal",
    "ugly, hideous": "ul-",
    "under": "di-, nu",
    "understand": "henia-",
    "understanding": "hannas",
    "union": "erthad",
    "unite": "ertha-",
    "unquenchable": "uluithiad",
    "untamed, wild": "rhaw",
    "uphill": "ambenn",
    "urge on, speed": "hortha-",
    "us": "ammen",
    "use": "iuith, iuitha-",
    "usual": "órui",
    "vague, obscure, foggy": "hethu",
    "Vala": "rodon",
    "valley": "imrath, lâd, nand",
    "valour": "gorn",
    "Vanya": "miniel",
    "veiled, hidden, shadowed": "hall",
    "vengeance": "acharn",
    "verse (poetry)": "tenn",
    "vessel (water)": "calph",
    "victor, winner": "degil",
    "vigour": "gôr",
    "vine, wine": "gwîn",
    "violent, sudden": "bregol",
    "virginity": "gweneth",
    "vision": "tírad",
    "void": "gaw",
    "wagon, wain": "rach",
    "wake up": "echuia-",
    "walk": "pada-",
    "wall": "ram, rammas",
    "warm": "laug",
    "warrior": "maethor",
    "watch": "tíra-",
    "watcher": "tirn",
    "waterfall": "espalass, lanthir",
    "watery": "nend, nínui",
    "weary": "lom",
    "weaver, webster": "nathron",
    "web": "nath",
    "web, net": "gwi",
    "Wednesday (moon-day)": "orithil",
    "week of five days": "lefnor",
    "well": "mae",
    "werewolf": "gaur",
    "werewolf-host": "gaurhoth",
    "west": "annû-, dúven",
    "West": "dûn",
    "West, Sunset": "annûn",
    "western": "annui",
    "Westron": "Annúnaid",
    "wet": "limp, mesc, mesg, nîn",
    "when": "ir",
    "where": "ias",
    "white": "fain",
    "white heat": "brass",
    "white-hot": "brassen",
    "whiten": "nimmida-",
    "who? what?": "man?",
    "whole, complete, full": "pant",
    "why?": "amman?",
    "wide": "pann",
    "wife": "hervess",
    "wild man": "drû",
    "Wild Man": "Drúadan, rhavan",
    "Wild Men": "Drúath",
    "wilderness": "rhovan",
    "wildfire": "bregedúr",
    "will": "innas, thel-?",
    "will not": "ava-",
    "willow": "tathar, tathor",
    "window": "henneth",
    "windy": "gwaeren",
    "winter": "rhîw",
    "wisdom": "golu",
    "wise": "goll, sael",
    "wish": "iest",
    "with, at, by near": "na",
    "without": "pen",
    "wizard": "curunír, ithron",
    "wolf": "araf, draug, garaf",
    "wolf-howl": "gaul",
    "woman, wife": "bess",
    "wooden": "tawaren",
    "woodpecker": "tavor",
    "wool": "taw",
    "word": "peth",
    "world": "amar",
    "wound": "harna-, haru",
    "wounded": "harn",
    "wrist": "molif",
    "write": "teitha-",
    "yelling-horde": "glamhoth",
    "yellow": "malen",
    "yellow bird, yellowhammer": "emlin",
    "young": "neth",
    "young maiden": "dess",
    "youth": "nîth",
    "May they find peace after death.": "Hiro hyn hîdh ab 'wanath.",
    "Something's out there!": "Aragorn, nad no ennas!",
    "What do you see?": "Man cenich?",
    "You told me once… this day would come.": "Min lû pedich nin… i aur hen telitha.",
    "This is not the end… it is the beginning. You must go with Frodo. This is your path.": "Ú i vethed… nâ i onnad. Boe bedich go Frodo. Han bâd lîn.",
    "My path is hidden from me": "Dolen i vâd o nin.",
    "It is already laid before your feet. You cannot falter now.": "Si peliannen i vâd na dail lîn. Si boe ú-dhannathach.",
    "If you trust nothing else… trust this… trust us.": "Ae ú-esteliach nad… estelio han… estelio ammen.",
    "Is this how you would take your leave? Did you think you could slip away at first light - unnoticed?": "Nach gwannatha sin? Ma nathach hi gwannathach or minuial archened?",
    "I will not be coming back.": "Ú-ethelithon.",
    "You underestimate your skill in battle… You will come back.": "Estelio guru lîn ne dagor… Ethelithach.",
    "It is not of death in battle that I speak.": "Ú-bedin o gurth ne dagor.",
    "What do you speak of?": "O man pedich?",
    "You have a chance for another life, away from war… grief… despair.": "Edra le men, men na guil edwen, haer o auth… a nîr… a naeth.",
    "The ships are leaving for Valinor. Go now, before it is too late.": "Tollen i lû. I chair gwannar na Valannor. Si bado, no círar.",
    "Do I not also have your love?": "Ah im, ú-'erin veleth lîn?",
    "You have my love, daddy.": "Gerich veleth nîn, ada.",
    "The world is changed; I can feel it in the water, I can feel it in the earth, I can smell it in the air.": "I amar prestar aen, han mathon ne nen, han mathon ne chae a han noston ned 'wilith.",
    "What troubles you, Brego? What do you see?": "Man le trasta, Brego. Man cenich?",
    "Well done, Brego, my friend.": "Mae carnen, Brego, mellon nîn.",
    "You're late.": "Le abdollen.",
    "Thank you.": "Hannon le.",
    "And they should be… three hundred against ten thousand.": "Boe a hyn neled herain dan caer menig.",
    "They have more hope of defending themselves here than at Edoras.": "Si, beriathar hyn ammaeg na ned Edoras.",
    "They cannot win this fight. They are all going to die!": "Aragorn, nedin dagor hen ú-'erir ortheri. Natha daged dhaer.",
    "There is nothing to forgive": "Ú-moe edaved, Legolas",
    "Welcome, Haldir.": "Mae govannen, Haldir.",
    "Oh Children of Eru, Show them no mercy, for you shall receive none.": "A Eruchîn, ú-dano i faelas a hyn an uben tanatha le faelas.",
    "Hold!": "Dartho!",
    "Prepare to fire": "Tangado a chadad!",
    "Their armour is weak at the neck… and beneath the arm.": "Faeg i-varv dîn na lanc a nu ranc!",
    "Release arrows!": "Hado i philinn!",
    "Ladders!": "Pendraith!",
    "Bring him down, Legolas! Kill him!": "Togo hon dad, Legolas! Dago hon!",
    "Causeway!": "Na fennas!",
    "Shoot continuously!": "Hado ribed!",
    "Throw!": "Hado!",
    "Charge!": "Herio!",
    "To the Keep!": "Nan barad!"
}

elvish_eval = {
    "May they find peace after death.": "Hiro hyn hîdh ab 'wanath.",
    "Something's out there!": "Aragorn, nad no ennas!",
    "What do you see?": "Man cenich?",
    "You told me once… this day would come.": "Min lû pedich nin… i aur hen telitha.",
    "This is not the end… it is the beginning.": "Ú i vethed… nâ i onnad.",
    "My path is hidden from me": "Dolen i vâd o nin.",
    "It is already laid before your feet.": "Si peliannen i vâd na dail lîn.",
    "If you trust nothing else… trust this… trust us.": "Ae ú-esteliach nad… estelio han… estelio ammen.",
    "I will not be coming back.": "Ú-ethelithon.",
    "You underestimate your skill in battle.": "Estelio guru lîn ne dagor.",
    "It is not of death in battle that I speak.": "Ú-bedin o gurth ne dagor.",
    "What do you speak of?": "O man pedich?",
    "The ships are leaving for Valinor.": "I chair gwannar na Valannor.",
    "Do I not also have your love?": "Ah im, ú-'erin veleth lîn?",
    "You have my love, daddy.": "Gerich veleth nîn, ada.",
    "The world is changed.": "I amar prestar aen.",
    "What troubles you, Brego?": "Man le trasta, Brego.",
    "Well done, Brego, my friend.": "Mae carnen, Brego, mellon nîn.",
    "You're late.": "Le abdollen.",
    "Thank you.": "Hannon le.",
    "There is nothing to forgive": "Ú-moe edaved, Legolas",
    "Welcome, Haldir.": "Mae govannen, Haldir.",
    "Hold!": "Dartho!",
    "Prepare to fire": "Tangado a chadad!",
    "Release arrows!": "Hado i philinn!",
    "Ladders!": "Pendraith!",
    "Bring him down, Legolas!": "Togo hon dad, Legolas!",
    "Causeway!": "Na fennas!",
    "Throw!": "Hado!",
    "Charge!": "Herio!",
    "To the Keep!": "Nan barad!",
    "I speak": "Im ped-",
    "The king comes": "I aran tol-",
    "A friend runs": "Mellon nor-",
    "The star shines": "I êl síla-",
    "I see the ship": "Im cen- i cair",
    "The fire is hot": "I naur na born",
    "Night falls": "Dû danna-",
    "The leaf falls": "I lass danna-",
    "The river flows": "I sirion siria-",
    "He sleeps": "Hon losta-",
    "She eats bread": "He mad- bass",
    "The dog runs": "I hû nor-",
    "The sword is sharp": "I megil na laeg",
    "Farewell, king": "Navaer, aran",
    "The day is long": "I aur na and",
    "Cold water": "Nên ring",
    "The tree grows": "I galadh gala-",
    "I drink wine": "Im soga gwîn",
    "The moon shines": "Ithil síla-",
    "The sun is gold": "Anor na mall"
}

In [ ]:
#@title **Dothraki Train-Test Data**

dothraki_phrases = {
    "Respect to one that is respectful, great khal.": "Athchomar chomakaan, zhey khal vezhven",
    "May I present my honored guests, Viserys… and his sister Daenerys.": "Azha anhaan asshilat...",
    "Come, Jorah the Andal.": "Jadi, zhey Jorah Andahli",
    "Great khal.": "Khal vezhven",
    "Tonight I would look upon your face.": "Ajjalan anha zalat vitiherat yer hatif",
    "Only while blood of my blood lives.": "Kash qoy qoyi thira disse",
    "Stop the horde.": "Annakhas dozgosores",
    "Do you want him dead, khaleesi?": "Hash shafka zali addrivat mae, zhey Khaleesi?",
    "Maybe the ear will listen if it is removed from the head.": "Ishish chare acharoe hash me nem ejervae nharesoon",
    "Khaleesi does not want him harmed.": "Khaleesi vos zalo meme nem azisa",
    "Cricket.": "Chiftik",
    "Pride, prowess.": "Athjahakar",
    "It's a blessing from the great stallion.": "Me azhasavva vezhofoon",
    "Dothraki don't wear steel dresses.": "Dothraki vos ondeo shoris tawakofi",
    "Slow.": "Vroz",
    "My father taught me how to fight. He taught me that speed defeats size.": "Ave anni ezzo anhaan os lajataan. Me ezzo anhaan m'athdikar assie athzhokwazar",
    "I heard that your father was a famous warrior.": "Anha ray char m'ave yeri lajakoon hakeso",
    "He was bloodrider to Khal Bharbo.": "Me dothrakhqoyoon torga Khali Bharbo",
    "And your father, Jorah the Andal? He was a warrior also?": "Majin ave yeri Jorah Andahli? Hash me lajakoon akka?",
    "The khaleesi wants to eat something different tonight.": "Khaleesi zala meme adakha esinakh ajjalan",
    "There are no rabbits.": "Vo mawizzi vekho jinne",
    "Find some ducks for me. She likes ducks.": "Ezas loy alegri h'anhaan. Mori allayafi mae, jin alegra",
    "Have you seen any ducks, woman? No rabbits, no ducks. Do you have eyes in your head?": "Hash yer ray tih loy alegra zhey vikeesi? Vo mawizzi, vo alegri. Hash tihi vekhi she nhare yeroon?",
    "Dogs then. I have seen many dogs.": "Majin jano. Anha ray tih san jani",
    "A blessing from the great stallion.": "Azhasavva vezhofoon",
    "It's a boy.": "Me rakh",
    "How do you know?": "Kifinosi yer nesi?",
    "I very knows.": "Anha sekke nesa",
    "to silence": "acchakat",
    "smelly": "achra",
    "stink": "achrakh",
    "to be smelly": "achralat",
    "to eat": "adakhat",
    "to feed on": "adakhilat",
    "to feed": "addakhat",
    "midnight": "addo ajjalani",
    "to convey": "addothralat",
    "to cause to spill": "addrekat",
    "to kill": "addrivat",
    "chair": "ador",
    "turtle": "adra",
    "morning": "aena",
    "hot": "afazh",
    "warm": "afazhi",
    "whoa": "affa",
    "shh": "affa",
    "to warm": "affazhat",
    "to replace": "affazolat",
    "to make someone itch": "affesat",
    "when": "affin",
    "to clean": "affisat",
    "fly": "afis",
    "second": "akataki",
    "two hundred": "akatken",
    "twelve": "akatthi",
    "also": "akka",
    "both": "akkate",
    "leader": "akkelenak",
    "to decide": "akkelenat",
    "stand up": "akko",
    "to stand something up": "akkovarat",
    "some": "ale",
    "duck": "alegra",
    "more": "alikh",
    "to please": "allayafat",
    "farther": "alle",
    "to pledge": "alloshat",
    "to decorate": "ammasat",
    "to hobble a person": "ammattelat",
    "to play a musical instrument": "ammemat",
    "to empty": "ammenat",
    "to rest": "ammithrat",
    "to jog": "anat",
    "I": "anha",
    "me": "anna",
    "to rip": "aggendat",
    "snow": "ahesh",
    "winter": "aheshke",
    "to frustrate": "ahhaqat",
    "to sharpen": "ahhasat",
    "then": "ahhaz",
    "yes": "ai",
    "tonight": "ajjalan",
    "now": "ajjin",
    "two": "akat",
    "second (ordinal)": "akatak",
    "to stop": "annakhat",
    "to put an end to": "annakholat",
    "announcement": "annese",
    "to sit something": "annevalat",
    "of mine": "anni",
    "to cause pain": "annithat",
    "to encourage": "annithilat",
    "deep": "ao",
    "dark": "ao",
    "to keep near": "aqqisat",
    "curved sword": "arakh",
    "to drop": "aranat",
    "subject": "aranikh",
    "to be neglected": "arannat",
    "carelessly": "aranne",
    "coward": "aresak",
    "to choke": "arraggat",
    "first": "ataki",
    "dry": "ath",
    "fatality": "athaddrivar",
    "frustration": "athahhaqar",
    "depths": "athaozar",
    "wasteland": "athasar",
    "to shame": "arranat",
    "shaming": "arrane",
    "then (future)": "arrek",
    "until": "arrekaan",
    "from since": "arrekoon",
    "to fix": "arrissat",
    "to frighten": "arrokhat",
    "to fall": "arthasat",
    "sky": "asavva",
    "word": "ase",
    "cranberry": "asfavirzeth",
    "river": "ashefa",
    "oath": "asqoyi",
    "to break": "assamvat",
    "today": "asshekh",
    "day": "asshekh",
    "birthday": "asshekhqoyi",
    "introduction": "asshie",
    "to introduce": "asshilat",
    "sign": "assikhqoyi",
    "prophecy": "assikhqoyisir",
    "to signal": "assilat",
    "to defeat": "assilat",
    "message": "assokh",
    "to command": "assolat",
    "to say": "astat",
    "to joke": "astilat",
    "saying": "asto",
    "to speak": "astolat",
    "one": "at",
    "first (ordinal)": "atak",
    "nonsense": "athastokhdeveshizar",
    "waiting": "athayozar",
    "limit": "athchilar",
    "respect": "athchomar",
    "respect to those that are respectful": "athchomar chomakea",
    "lameness": "athdarinar",
    "usefulness": "athdavrazar",
    "speed": "athdikar",
    "comfort": "athdisizar",
    "death": "athdrivar",
    "sunset": "athdrivar shekhi",
    "nightmare": "athdrivarido",
    "return": "athessazar",
    "departure": "athezar",
    "dancing": "athezhirar",
    "love (platonic)": "athfiezar",
    "certainty": "athgoshar",
    "strength": "athhajar",
    "sex": "athhilezar",
    "woman-on-top sex": "athijezar",
    "pride": "athjahakar",
    "discussion": "athjerizar",
    "correctness": "athjilar",
    "marriage": "athkemar",
    "running": "athlanar",
    "wailing": "athlaqar",
    "loudness": "athlavakhar",
    "anxiousness": "athmanimvenar",
    "ache": "athmharar",
    "rest": "athmithrar",
    "magic": "athmovezar",
    "rigidity": "athniqar",
    "pain": "athnithar",
    "painlessness": "athnithmenar",
    "collapse": "athohharar",
    "to dry": "atholat",
    "fury": "athostar",
    "fear": "athrokhar",
    "vision": "athtihar",
    "greatness": "athvezhvenar",
    "war": "athvilajerar",
    "wisdom": "athvillar",
    "birth": "athyolar",
    "hope": "athzalar",
    "sickness": "athzhikhar",
    "love (romantic)": "athzhilar",
    "size": "athzhokwazar",
    "danger": "athzhowakar",
    "something": "ato",
    "firstly": "atte",
    "duty": "atthar",
    "to destroy": "atthasat",
    "eleven": "atthi",
    "life": "atthirar",
    "dream": "atthirarido",
    "to show": "attihat",
    "father": "ave",
    "to make something kneel": "avvemolat",
    "to burn something": "avvirsalat",
    "to grow something": "avvitisherat",
    "never": "avvos",
    "screamer": "awazak",
    "to scream": "awazat",
    "bell": "ayena",
    "to wait": "ayolat",
    "to lift": "ayyathat",
    "always": "ayyey",
    "blade": "az",
    "blessing": "azhasavva",
    "to give": "azhat",
    "gift": "azho",
    "slaver": "azzafrok",
    "to enslave": "azzafrolat",
    "to harm": "azzisat",
    "to put down": "azzohat",
    "wind": "chaf",
    "mercenary": "chafak",
    "climate": "chafasar",
    "autumn": "chafka",
    "silent": "chak",
    "twenty": "chakat",
    "bean": "chal",
    "to hear": "charat",
    "ear": "chare",
    "to listen": "charolat",
    "either": "che",
    "well": "chek",
    "very well": "chekosshi",
    "locust": "chelsian",
    "liver": "cheno",
    "half mile": "chetira",
    "to canter": "chetirat",
    "dark bay horse": "cheyao",
    "dark brown skinned": "cheyaoven",
    "seventy": "chifekh",
    "cricket": "chifti",
    "weak hit": "chiftikh",
    "to lie down": "chilat",
    "lying": "chilay",
    "fifty": "chimek",
    "woman": "chiori",
    "wife": "chiorikem",
    "ninety": "chiqazat",
    "almost": "chir",
    "thirty": "chisen",
    "forty": "chitor",
    "horn": "chiva",
    "sixty": "chizhinda",
    "to be bitter": "choakat",
    "one who is respectful": "chomak",
    "to be respectful": "chomat",
    "honor": "chomokh",
    "to grow respectful": "chomolat",
    "to be hard": "chongat",
    "to solidify": "chongolat",
    "pika": "choo",
    "eighty": "chori",
    "to roll": "chorkat",
    "fresh": "chosh",
    "beet": "chot",
    "butt": "choyo",
    "leaf": "daeni",
    "green": "dahaan",
    "type of grass": "dahana",
    "pomegranate": "daki",
    "thousand": "dalen",
    "cow": "dalfe",
    "ford": "dan",
    "gem": "dan",
    "clearing": "dana",
    "saddle": "darif",
    "lame": "darin",
    "rosemary bush": "dave",
    "pungent": "dave",
    "good": "davra",
    "useful thing": "davrakh",
    "to be useful": "davralat",
    "rosemary": "davveya",
    "cheek": "dech",
    "shallow": "dei",
    "light (color)": "dei",
    "fog": "devesh",
    "to know how": "devolat",
    "skull": "diaf",
    "fast": "dik",
    "flash": "dili",
    "to pass": "dinat",
    "to think": "dirgat",
    "thought": "dirge",
    "simple": "dis",
    "bark": "dishah",
    "very simple": "disisse",
    "only": "disse",
    "wet": "diwe",
    "to be wet": "diwelat",
    "to suffer": "dogat",
    "to shout": "donat",
    "goat": "dorvi",
    "ibex": "dorvof",
    "council": "dosh",
    "rider": "dothrak",
    "ride": "dothrakh",
    "bloodrider": "dothrakhqoyi",
    "to ride": "dothralat",
    "animal carcass": "dozgikh",
    "enemy": "dozgo",
    "enemy horde": "dozgosor",
    "nursing mother": "drane",
    "mother's milk": "dranekh",
    "to spill": "drekat",
    "to tip over": "drekolat",
    "dead": "driv",
    "to be dead": "drivat",
    "to die": "drivolat",
    "to drive animals": "drogat",
    "herd": "drogikh",
    "to slay": "drozhat",
    "useless": "edavrasa",
    "to squeeze": "efat",
    "shackle": "efe",
    "to uproot": "efesalat",
    "to disagree": "efichisalat",
    "all": "ei",
    "bank of river": "ejakh",
    "to remove": "ejervalat",
    "to undress": "ekhogaralat",
    "seed": "elain",
    "to plant": "elainat",
    "to go": "elat",
    "via point": "eleisosakh",
    "shoulder": "elme",
    "to respond": "elzat",
    "response": "elzikh",
    "to smile": "emat",
    "smile": "eme",
    "smiley": "emesh",
    "to make something enter": "emmalat",
    "to amuse": "emmat",
    "gate": "emrakh",
    "to enter": "emralat",
    "mule": "enossho",
    "baby": "enta",
    "to let go of": "eqorasalat",
    "kind": "erin",
    "lady": "erinak",
    "to be kind": "erinat",
    "old": "ershe",
    "smooth": "es",
    "to take something back": "esazhalat",
    "exit": "esemrasakh",
    "to exit": "esemrasalat",
    "to get up": "eshat",
    "fish": "eshina",
    "another": "eshna",
    "different": "esina",
    "different thing": "esinakh",
    "to be different": "esinalat",
    "to change": "esinasolat",
    "braggart": "esittesak",
    "to brag": "esittesalat",
    "to return": "essalat",
    "roof": "essheya",
    "must": "eth",
    "to start doing": "evat",
    "tail": "eve",
    "water": "eveth",
    "poison water": "evethiz",
    "to start": "evolat",
    "all along": "evoon",
    "to start something": "evvat",
    "olive": "ewe",
    "olive oil": "ewekh",
    "olive pit": "ewweya",
    "everyone": "eyak",
    "rain": "eyel",
    "to move something": "eyelat",
    "spotted": "eyeli",
    "spring": "eyelke",
    "to find": "ezat",
    "to dance": "ezhirat",
    "dancing (adj)": "ezhiray",
    "learner": "ezok",
    "to learn": "ezolat",
    "to teach": "ezzolat",
    "to kick": "fakat",
    "dappled horse": "fansa",
    "corn bunting": "faqqi",
    "cloud": "fas",
    "destiny": "fasqoyi",
    "to slap": "fatat",
    "to insult": "fatilat",
    "to hate": "fejat",
    "seven": "fekh",
    "seventh": "fekhak",
    "seven hundred": "fekhken",
    "seventeen": "fekhthi",
    "wing": "felde",
    "feather": "feldekh",
    "to attach": "fenat",
    "hemp": "ferri",
    "tree": "feshith",
    "carrot": "feso",
    "to thirst": "fevelat",
    "to fetch": "fichat",
    "rope": "fiez",
    "smoke": "fih",
    "coward (filkak)": "filkak",
    "to move like a caterpillar": "filkat",
    "caterpillar": "filki",
    "dried plum": "filla",
    "who": "fin",
    "what": "fini",
    "where": "finne",
    "round": "fir",
    "ring": "fire",
    "year": "firesof",
    "ball": "firi",
    "crown": "firikhnharen",
    "cold": "fish",
    "short": "fitte",
    "film": "flas",
    "update": "flasikh",
    "dull": "flech",
    "hunter": "fonak",
    "hunting party": "fonakasar",
    "hunt well": "fonas chek",
    "to hunt": "fonat",
    "advice": "fonnoya",
    "to track": "fonolat",
    "throat": "fotha",
    "old (foz)": "foz",
    "elder": "fozak",
    "to touch": "frakhat",
    "to feel": "frakholat",
    "sound of horse's hooves": "fredri",
    "chatterbox": "fredrik",
    "to solve": "gachat",
    "place": "gache",
    "lungs": "gadim",
    "brother": "gaezo",
    "egg": "gale",
    "rice": "gamiz",
    "to lean": "ganat",
    "belly": "gango",
    "intestines": "gaoli",
    "root": "garfoth",
    "to grow hungry": "garvolat",
    "meat": "gavat",
    "bolas": "gehqoyi",
    "to be ripped": "gendat",
    "rip": "gende",
    "ripped": "gende",
    "to lack": "gerat",
    "lack": "gerikh",
    "mole": "geve",
    "snake": "gezri",
    "decoy strike": "gezrikh",
    "to push": "gidat",
    "breeze": "gillo",
    "weather": "gillosor",
    "mouse": "gimi",
    "bee": "giz",
    "honey": "gizikh",
    "sweet": "gizikhven",
    "onion": "glas",
    "louse": "glen",
    "mouth": "gomma",
    "to charge": "gorat",
    "to be certain": "goshat",
    "to mate": "govat",
    "waste": "graddakh",
    "wall": "gref",
    "Let's go": "gwe",
    "food": "hadaen",
    "gust of wind": "hador",
    "spawn": "haesh",
    "soft": "haf",
    "strong": "haj",
    "right (direction)": "haje",
    "right side": "hajekh",
    "to grow strong": "hajolat",
    "name": "hake",
    "to name": "hakelat",
    "famous": "hakeso",
    "flower": "halah",
    "finally": "ha nakhaan",
    "hey": "hale",
    "yak": "hammi",
    "rose": "hanna",
    "to be tired": "haqat",
    "tired": "haqe",
    "exhausted": "haqeqqe",
    "sharp": "has",
    "if": "hash",
    "face": "hatif",
    "sea": "havazh",
    "to move weakly": "havolat",
    "bread": "havon",
    "cat": "havzi",
    "lazy": "havziven",
    "that": "haz",
    "there": "hazze",
    "jug": "heffof",
    "fur": "hem",
    "furry": "heme",
    "southern": "hesk",
    "south": "heske",
    "rim": "heth",
    "tight": "hethke",
    "far": "hezhah",
    "to dig": "hilelat",
    "to clench": "himat",
    "fist": "himo",
    "glove": "hlaka",
    "bear": "hlizif",
    "wrist": "hlofa",
    "to blow": "holat",
    "giddyup": "hosh",
    "golden": "hoshor",
    "mouth of an animal": "hoska",
    "to sing": "hoyalat",
    "song": "hoyali",
    "white lion": "hrakkar",
    "grass": "hranna",
    "horse": "hrazef",
    "wild horses": "hrazefeser",
    "to welcome": "iddelat",
    "owl": "idiro",
    "wooden": "ido",
    "leader of the hunt": "idrik",
    "to guide": "idrilat",
    "foreigner": "ifak",
    "to walk": "ifat",
    "victory (iffi)": "iffi",
    "bucket": "igge",
    "to swallow": "ijelat",
    "ashes": "ikh",
    "to strike": "ildat",
    "sword strike": "ildo",
    "skin": "ilek",
    "vegetable": "ilme",
    "salad": "ilmeser",
    "young": "imesh",
    "belt": "imo",
    "sister": "inavva",
    "to drink": "indelat",
    "beetle": "inte",
    "back": "irge",
    "trout": "irra",
    "to trot": "irvosat",
    "maybe": "ishish",
    "well proven": "iste",
    "suitably warm": "ita",
    "brittle": "ithkoil",
    "to test": "ittelat",
    "wild": "ivezh",
    "beast": "ivezho",
    "to melt": "ivisat",
    "to be poisonous": "izat",
    "poison": "ize",
    "forbidden": "izven",
    "to poison": "izzat",
    "to come": "jadat",
    "to approach": "jadilat",
    "to arrive": "jadolat",
    "vulture": "jadro",
    "braid": "jahak",
    "moon": "jalan",
    "blanket": "janha",
    "niece": "janise",
    "dog": "jano",
    "executioner": "jaqqa",
    "to laugh": "jasat",
    "reins": "javrath",
    "to block": "jazat",
    "pony": "jedda",
    "to bounce": "jekhtat",
    "lemon": "jela",
    "sour": "jelaven",
    "cheese": "jelli",
    "trader": "jerak",
    "wine merchant": "jerak sewafikhaan",
    "to trade": "jerat",
    "to imprison": "jervat",
    "to pull": "jesat",
    "market": "jeser",
    "ice": "jesh",
    "frozen": "jesho",
    "fox": "jith",
    "chicken": "jiz",
    "here": "jinne",
    "to cook": "jolinat",
    "cooking pot": "jolino",
    "salmon": "joma",
    "closed": "jon",
    "idol": "joro",
    "corn": "jorok",
    "to capture": "kadat",
    "net": "kade",
    "to smash": "kafat",
    "pestle": "kafe",
    "to crush": "kaffat",
    "lark": "kane",
    "mile": "karlina",
    "to gallop": "karlinat",
    "to pull with both hands": "kartat",
    "while": "kash",
    "to last": "kashat",
    "time": "kashi",
    "black": "kazga",
    "patterned": "kelen",
    "married": "kem",
    "spouse": "kemak",
    "to marry": "kemat",
    "ally": "kemik",
    "dried fig": "kemis",
    "opal": "kendra",
    "stick": "kerikh",
    "to carry": "kesselat",
    "bodyguard": "kha",
    "body": "khado",
    "corpse": "khadokh",
    "king": "khal",
    "prince": "khalakka",
    "princess": "khalakki",
    "horde": "khalasar",
    "queen": "khaleesi",
    "waist": "khaor",
    "to sneeze": "khefat",
    "flesh": "kher",
    "leather": "kherikh",
    "worm": "khewo",
    "to be sad": "khezhat",
    "flag": "khiro",
    "marsh": "khizra",
    "clothes": "khogar",
    "to dress": "khogarat",
    "box": "khogari",
    "because of": "ki",
    "why": "kifindirgi",
    "how": "kifinosi",
    "ancestor": "kim",
    "original": "kim",
    "ancestry": "kimisir",
    "we": "kisha",
    "our": "kishi",
    "medicine": "koala",
    "healer": "koalak",
    "nick": "koge",
    "flawless": "kogmen",
    "bow": "kohol",
    "to cure": "kolat",
    "eagle": "kolver",
    "to stand": "kovarat",
    "mountain": "krazaaj",
    "grandmother": "kristasof",
    "flea": "krol",
    "beautiful (inanimate)": "lain",
    "warrior": "lajak",
    "army": "lajasar",
    "to fight": "lajat",
    "to chew": "lakkhat",
    "mare": "lame",
    "to run": "lanat",
    "killing spree": "lanqoyi",
    "to cry": "laqat",
    "tear": "laqikh",
    "stew": "lashfak",
    "loud": "lavakh",
    "to be happy": "layafat",
    "ghost": "lei",
    "lost": "lei",
    "orphan": "leishak",
    "language": "lekh",
    "tongue": "lekh",
    "enough": "lekhaan",
    "taste": "lekhi",
    "to taste": "lekhilat",
    "neck": "lenta",
    "rat": "leqse",
    "to breathe": "leshitat",
    "tattoo": "lir",
    "tattoo artist": "lirak",
    "to tattoo": "lirat",
    "to hit": "lojat",
    "to bathe": "lommat",
    "arrow": "loqam",
    "to wink": "lorat",
    "to blink": "lorrat",
    "guard": "loshak",
    "disappointed": "lost",
    "puddle": "loy",
    "bad": "mel",
    "to be evil": "melat",
    "sound": "mem",
    "to make a sound": "memat",
    "to be empty": "menat",
    "to be swollen": "mesat",
    "to be new": "meshat",
    "to be pregnant": "mesilat",
    "soup": "mesina",
    "muscle": "meso",
    "to swell": "mesolat",
    "pale skinned": "messhihven",
    "sexy": "mezahe",
    "whore": "mezhah",
    "headache": "mhari",
    "thorn": "mihe",
    "dagger": "mihesof",
    "to rest": "mithrat",
    "they": "mori",
    "sack": "moska",
    "magician": "movek",
    "to make by hand": "movelat",
    "within": "mra",
    "garlic": "mredi",
    "victorious": "najah",
    "winner": "najahak",
    "victory": "najaheya",
    "completely": "nakhaan",
    "to stop": "nakhat",
    "small": "naqis",
    "to urinate": "navat",
    "beginner": "navik",
    "ashamed": "navvirzethay",
    "girl": "nayat",
    "long": "neak",
    "stone": "negwin",
    "foal": "nerro",
    "to know": "nesat",
    "information": "nesikh",
    "guest": "nevak",
    "to sit": "nevalat",
    "head": "nhare",
    "raven": "nhizo",
    "sausage": "nindi",
    "blood sausage": "ninthqoyi",
    "rigid": "niqe",
    "orgasm": "niqikkheya",
    "to be full": "nirat",
    "painful": "nith",
    "together": "niyanqoy",
    "to be tough": "nizhat",
    "bull": "noah",
    "to wipe": "nofilat",
    "to trample": "nokittat",
    "hair": "noreth",
    "to turn": "notat",
    "chestnut horse": "nozho",
    "straight": "nris",
    "thick": "nroj",
    "soul": "oakah",
    "dun horse": "ocha",
    "breast": "odaya",
    "yam": "oeta",
    "to slaughter": "ogat",
    "daughter": "ohara",
    "to fall": "oharat",
    "heavy": "ohazh",
    "fermented": "ohazho",
    "to be still": "oholat",
    "fat": "oiro",
    "wrong": "ojil",
    "friend": "okeo",
    "to ring": "okhat",
    "to choose": "okkat",
    "tent": "okre",
    "stone house": "okrenegwin",
    "to rub": "oldat",
    "over": "oleth",
    "hill": "olta",
    "without": "oma",
    "to wear": "ondelat",
    "sheep": "oqet",
    "beat": "oqo",
    "heartbeat": "oqooqo",
    "eight": "ori",
    "eighth": "orik",
    "eighteen": "oritthi",
    "whip": "orvik",
    "shoe": "orzi",
    "trap": "orzo",
    "road": "os",
    "to suck": "oshat",
    "yesterday": "oskikh",
    "bridge": "osoleth",
    "to bite": "ostat",
    "wide": "ovah",
    "to fly": "ovethat",
    "opening": "ovrakh",
    "loose": "ovray",
    "to throw": "ovvethat",
    "ruby": "ozzir",
    "firefly": "qacha",
    "question": "qaf",
    "to ask": "qafat",
    "palimino horse": "qahlan",
    "stork": "qana",
    "to float": "qayelat",
    "nine": "qazat",
    "ninth": "qazatak",
    "apple": "qazer",
    "strange": "qemmemmo",
    "cover": "qemmo",
    "eyelid": "qemmotihan",
    "sand": "qeshah",
    "basket": "qeso",
    "forest": "qevir",
    "pig": "qifo",
    "island": "qile",
    "near": "qisi",
    "bleeding": "qiya",
    "deer": "qlaseh",
    "hand": "qora",
    "to seize": "qoralat",
    "prize": "qorasokh",
    "to rape": "qorasolat",
    "forearm": "qorraya",
    "spider": "qosar",
    "deceptive": "qosarven",
    "lie": "qosarvenikh",
    "repetitive": "qoth",
    "loyal": "qothat",
    "trembling": "qov",
    "blood": "qoy",
    "to vomit": "qwehat",
    "tiger": "rachel",
    "bright": "rahsan",
    "boy": "rakh",
    "land": "ramasar",
    "already": "ray",
    "summit": "rayan",
    "organ": "rea",
    "purple": "reaven",
    "thin": "redda",
    "skinny": "reddi",
    "asleep": "remek",
    "to sleep": "remekat",
    "hollow": "renrenoh",
    "stay": "reri",
    "foot": "rhae",
    "athlete": "rhaek",
    "world": "rhaesheser",
    "stride": "rhaesof",
    "cart": "rhaggat",
    "ship": "rhaggat eveth",
    "merciful": "rhanat",
    "to help": "rhelalat",
    "stirrup": "rhiko",
    "animal": "rhoa",
    "family": "rhojosor",
    "bubble": "rich",
    "raisin": "rigwa",
    "rotten": "rikh",
    "to cut": "rissat",
    "nose": "riv",
    "to smell": "rivvat",
    "son": "rizh",
    "to scrape": "rochat",
    "shriveled": "rof",
    "to fear": "rokhat",
    "to split": "sachat",
    "half": "sachi",
    "rider": "sajak",
    "to mount": "sajat",
    "steed": "sajo",
    "broken": "samva",
    "numerous": "samven",
    "heap": "san",
    "much": "san",
    "safe": "sandi",
    "to count": "sanneyalat",
    "new": "sash",
    "juicy": "sathoma",
    "orange": "sathomakh",
    "again": "save",
    "to be protected": "savidosalat",
    "yes": "sek",
    "too much": "sekke",
    "surely": "sekosshi",
    "three": "sen",
    "third": "senak",
    "three hundred": "senken",
    "thirteen": "senthi",
    "free": "seris",
    "vest": "serja",
    "grape": "sewaf",
    "wine": "sewafikh",
    "you (respectful)": "shafka",
    "healthy": "sharat",
    "sun": "shekh",
    "setting sun": "shekh drivo",
    "light": "shekhikh",
    "star": "shierak",
    "comet": "shierak qiya",
    "to snore": "shigat",
    "snore": "shigi",
    "gossip": "shilak",
    "to know a person": "shilat",
    "to trust": "shillat",
    "to believe": "shillolat",
    "to meet": "shilolat",
    "tame": "shim",
    "to burn out": "shinat",
    "gray": "shiqeth",
    "iron": "shiqethi",
    "beard": "shirane",
    "scorpion": "shiro",
    "spot": "sho",
    "to jump over": "shokat",
    "to mark": "sholat",
    "dress": "shor",
    "armor": "shor tawakof",
    "marked": "shoven",
    "nephew": "siera",
    "to spit": "sikhtelat",
    "follower": "silak",
    "to flash": "silat",
    "to follow": "sillat",
    "next": "sille",
    "tomorrow": "silokh",
    "male relative": "simon",
    "grandfather": "simonof",
    "left": "sindarine",
    "lightning": "sisi",
    "zero": "som",
    "absent": "som",
    "dirty": "sorf",
    "dirt": "sorfo",
    "earth": "sorfosor",
    "halt": "soroh",
    "dragon glass": "sondra",
    "to weave": "soqat",
    "plum": "soqwi",
    "metal": "taoka",
    "silkworm": "taso",
    "silk": "tasokh",
    "to do": "tat",
    "chop": "tavat",
    "authentic": "tawak",
    "thunder": "temme",
    "yogurt": "thagwa",
    "soft": "thash",
    "marmot": "thave",
    "blue": "thelis",
    "nipple": "theya",
    "pink": "theyaven",
    "ten": "thi",
    "narrow": "thif",
    "tenth": "thik",
    "task": "thikh",
    "to leak": "thimalat",
    "alive": "thir",
    "to live": "thirat",
    "to dream": "thirat atthiraride",
    "to survive": "thirolat",
    "juice": "thom",
    "nut": "tif",
    "eye": "tih",
    "to look": "tihat",
    "view": "tihikh",
    "overview": "tihikhziri",
    "to look at": "tihilat",
    "to understand": "tiholat",
    "deed": "tikh",
    "verb": "tikkheya",
    "boot": "tim",
    "book": "timvir",
    "finger": "tir",
    "to point": "tirat",
    "nail": "tirikh",
    "eastern": "tith",
    "east": "titha",
    "dumb": "toki",
    "fool": "tokik",
    "bone": "tolorro",
    "four": "tor",
    "fourth": "torak",
    "stomach": "torga",
    "under": "torga",
    "four hundred": "torken",
    "fourteen": "torthi",
    "lake": "tozara",
    "city": "vaes",
    "to clog": "vaesilat",
    "lamb": "vafi",
    "wool": "vafikh",
    "horizon": "valad",
    "northern": "valsh",
    "north": "valshe",
    "stampede": "vash",
    "earthquake": "vash hrazefmen",
    "to converse": "vasterat",
    "to continue": "vatterat",
    "storm": "vaz",
    "stormborn": "vazyol",
    "to be present": "vekhat",
    "noun": "vekhikh",
    "to sow": "velainerat",
    "yellow": "veltor",
    "to stall": "velzerat",
    "reason why": "velzerikh",
    "knee": "vem",
    "like": "ven",
    "to be like": "venat",
    "class": "veneser",
    "wolf": "ver",
    "traveler": "verak",
    "caravan": "verakasar",
    "to travel": "verat",
    "violent": "verven",
    "stallion": "vezh",
    "horse lord": "vezhak",
    "great": "vezhven",
    "to shiver": "vichiterat",
    "respectful": "vichomer",
    "to miss someone": "vigererat",
    "to protect": "vijazerat",
    "to rescue": "vijazerolat",
    "chin": "vik",
    "to stay": "vikovarerat",
    "battle": "vilajero",
    "game": "vilajerosh",
    "wise": "ville",
    "to limp": "vimatterat",
    "to stab": "vindelat",
    "to remember": "vineserat",
    "portion of meat": "vinte",
    "to beg": "viqaferat",
    "to burn": "virsalat",
    "red": "virzeth",
    "red wine": "virzetha",
    "forehead": "vish",
    "to domesticate": "vishaferat",
    "color": "vishiya",
    "to flow": "vithalat",
    "to stare at": "vitiherat",
    "watch": "vitihirak",
    "to watch": "vitihirat",
    "to grow": "vitisherat",
    "feast": "vitteya",
    "silver": "vizhadi",
    "dust": "vod",
    "person": "voj",
    "god": "vojjor",
    "fire": "vorsa",
    "flame": "vorsakh",
    "summer": "vorsaska",
    "funeral pyre": "vorsqoyi",
    "tooth": "vorto",
    "not": "vos",
    "nothing": "vosi",
    "but": "vosma",
    "weapon": "vov",
    "to leap": "vrelat",
    "slow": "vroz",
    "nectarine": "yachi norethmen",
    "child": "yalli",
    "youth": "yamori",
    "collection": "yanqokh",
    "to collect": "yanqolat",
    "to flinch": "yarat",
    "up": "yath",
    "ramp": "yathokh",
    "to rise": "yatholat",
    "failure": "yeni",
    "you": "yer",
    "crone": "yesi",
    "frog": "yetto",
    "mad": "yofi",
    "born": "yol",
    "to be born": "yolat",
    "crossing": "yom",
    "to cross": "yomat",
    "across": "yomme",
    "million": "yor",
    "fruit": "yot",
    "brain": "yothnhare",
    "slave": "zafra",
    "to refuse": "zajjat",
    "to hope for": "zalat",
    "stable": "zan",
    "satchel": "zande",
    "shade": "zanisshi",
    "white": "zasqa",
    "to swim": "zerqolat",
    "butterfly": "zhalia",
    "spear": "zhani",
    "bass fish": "zhaqe",
    "lizard": "zhav",
    "dragon": "zhavorsa",
    "beautiful (animate)": "zheana",
    "to be beautiful": "zheanalat",
    "salt": "zhif",
    "salty": "zhifven",
    "to be sick": "zhikhat",
    "to love": "zhilat",
    "any": "zhille",
    "general": "zhilli",
    "advice": "zhillie",
    "six": "zhinda",
    "sixth": "zhindak",
    "six hundred": "zhindaken",
    "sixteen": "zhinthi",
    "enormous": "zhokakkwa",
    "big": "zhokwa",
    "to be large": "zhokwalat",
    "heart": "zhor",
    "own": "zhorre",
    "fig": "zhorrof",
    "dangerous": "zhowak",
    "disrespectful": "zichome",
    "to steal": "zifichelat",
    "to burden": "ziganelat",
    "to defy": "ziganesolat",
    "required": "zigere",
    "to require": "zigerelat",
    "to distract": "zimemelat",
    "remaining": "zinay",
    "to retch": "ziqwehelat",
    "bird": "zir",
    "to offend": "zireyeselat",
    "to cut off": "zirisselat",
    "to ache": "ziroqoselat",
    "raptor": "zirqoyi",
    "to be hurt": "zisat",
    "wound": "ziso",
    "scratch": "zisosh",
    "to fail someone": "ziyenelat",
    "low": "zoh",
    "down": "zohhe",
    "to be small": "zolat",
    "a little bit": "zolle",
    "kiss": "zoqwa",
    "to kiss": "zoqwat",
    "to roar": "zorat"
}

dothraki_eval = {
    "I ride.": "Anha dothralat.",
    "The horse is fast.": "Hrazef dik.",
    "The sword is sharp.": "Arakh has.",
    "My sun.": "Shekh anni.",
    "My stars.": "Shierak anni.",
    "Blood of my blood.": "Qoy qoyi.",
    "I see you.": "Anha tihat yer.",
    "You are beautiful.": "Yer zheana.",
    "It is known.": "Me nem nesa.",
    "No fear.": "Vos athrokhar.",
    "Eat the meat.": "Adakhas gavat.",
    "Drink wine.": "Indelas virzetha.",
    "Fire and blood.": "Vorsa ma qoy.",
    "Winter is coming.": "Aheshke jadat.",
    "The king rides.": "Khal dothralat.",
    "Protect the queen.": "Vijazeras khaleesi.",
    "Stop.": "Nakhas.",
    "Let's go.": "Gwe.",
    "I want to kill him.": "Anha zalat addrivat mae.",
    "He is a friend.": "Me okeo.",
    "She is my wife.": "Me chiorikem anni.",
    "He is my husband.": "Me mahrazhkem anni.",
    "Good morning.": "Aena davra.",
    "Much respect.": "San athchomar.",
    "Hello.": "M'ath.",
    "Hunt well.": "Fonas chek.",
    "Good birthday.": "Asshekhqoyi davra.",
    "The water is hot.": "Eveth afazh.",
    "The night is long.": "Ajjalan neak.",
    "I have a gift.": "Anha vekhat azho.",
    "Where is the horse?": "Finne hrazef?",
    "Who are you?": "Fin yer?",
    "What is that?": "Fini haz?",
    "I love you.": "Anha zhilat yer.",
    "Do not speak.": "Vos astas.",
    "Run fast.": "Lanas dik.",
    "Walk slowly.": "Ifas vroz.",
    "The dragon flies.": "Zhavorsa ovethat.",
    "Kill them all.": "Addrivas mori.",
    "Victory.": "Najaheya."
}


The below cell **`Initial LLM Call`** acts as the first call for initializaing the conversation with the LLM. It's responsible for the providing the example sequences for the translation demo and a general set of rules the LLM must follow.

In [ ]:
#@title **Initial LLM Call**
def initial_llm_call(language, examples_text):
  initial_prompt = f"""
      You are an expert translator specializing in a fictional movie language.

      You will be given:
      - A series of example translation pairs (English → fictional language)
      - A new English phrase to translate

      Your goals:
      - Infer the fictional language's rules from the examples
      - Use the vocabulary to translate the English Phrase
      - Translate the new English phrase with correct grammar and style
      - Maintain consistency with the patterns in the examples
      - Do NOT explain your reasoning or reveal the fictional language rules
      - Output **only** the translated phrase

      Examples:
      {examples_text}
  """
  chat.send_message(initial_prompt)

### **Let's Take a Look at an Example of the Prompt**

```
You are an expert translator specializing in a fictional movie language.

      You will be given:
      - A series of example translation pairs (English → fictional language)
      - A new English phrase to translate

      Your goals:
      - Infer the fictional language's rules from the examples
      - Use the vocabulary to translate the English Phrase
      - Translate the new English phrase with correct grammar and style
      - Maintain consistency with the patterns in the examples
      - Do NOT explain your reasoning or reveal the fictional language rules
      - Output **only** the translated phrase

      Examples:
      {examples}
```

This snippet shows an example of the initial warm-up prompt we pass to the LLM to prepare it for in-context learning. The prompt provides a clear instruction set, showing the model that it is acting as an expert translator for a fictional movie language along with a series of example English ↔ fictional-language pairs to illustrate the target patterns

The instructions explicitly tell the model to infer the rules and style from the examples, apply the correct vocabulary and grammar, and maintain consistency, while avoiding explanations or revealing any internal rules.

This warm-up prompt sets the context for the LLM, effectively priming it to perform in-context translation. By seeing the examples first, the model can generalize patterns and produce accurate translations for subsequent test phrases or user-provided inputs.

After the initial warm-up, we pass the LLM the actual translation prompt for each individual phrase. This prompt is sent in the form of a chat message and instructs the model to:
1. Act as an expert translator for the selected fictional language
2. Use the previously provided examples to guide its translation
3. Translate the specific English phrase provided.
4. Output only the translated phrase, without any extra punctuation, explanations, or commentary

```
You are an expert translator specializing in a fictional movie language.
    Using the previously provided examples,

    Translate:
    "{sample_english_phrase}"

    Output **only** the translated phrase, do not include any punctuation marks at the end of each phrase
```

This step focuses the LLM on producing the final translation based on the in-context examples. By separating the warm-up (pattern priming) from the specific phrase prompt, the model can generalize from the examples and generate accurate translations for unseen sentences.

The cell below `Demo (Predictions of Test Phrases)` is responsible for running the full translation-evaluation pipeline for the fictional movie languages. It serves as the end-to-end execution layer of the notebook, handling dataset selection, prompt construction, batchwise LLM inference, output parsing, and result storage.

The cell presents a dropdown menu for choosing one of the four fictional languages—Na’vi, Klingon, Elvish, or Dothraki—and automatically loads the corresponding training and test datasets. After the user selects a language, the system formats the example phrase pairs into a structured prompt and feeds them to the LLM.

The function performs the following steps, it first displays a dropdown allowing the user to select one of the four fictional languages. it then loads the associated training-example dictionary and the evaluation dataset and formats all example pairs into a consistent, line-by-line translation prompt. To start the inference process, we send an initial warm-up prompt to the LLM to prime the translation context. It will iterate through the test dataset in batches, calling the LLM for each English phrase. Lastly, we extract and clean the model output to obtain the predicted translation and stores the (input, expected, predicted) triples in a results table for easy access.

In [ ]:
#@title **Demo (Predictions of Test Phrases)**
chat = client.chats.create(model=model_name)
df_global = None

dropdown = widgets.Dropdown(
    options=["Select a language", "Na'vi", "Klingon", "Elvish", "Dothraki"],
    value="Select a language",
    description="Language:"
)

button = widgets.Button(
    description="Confirm",
    disabled=True
)

output = widgets.Output()

def update_button(*args):
    button.disabled = (dropdown.value == "Select a language")

dropdown.observe(update_button, 'value')

def build_example_string(language):
    if language == "Na'vi":
        data = navi_phrases
    elif language == "Klingon":
        data = klingon_phrases
    elif language == "Elvish":
        data = elvish_phrases
    elif language == "Dothraki":
        data = dothraki_phrases
    else:
        return ""
    return "\n".join(f"\"{eng}\" >> \"{trans}\"" for eng, trans in data.items())

def load_eval_dataset(language):
    if language == "Na'vi":
        return navi_eval
    elif language == "Klingon":
        return klingon_eval
    elif language == "Elvish":
        return elvish_eval
    elif language == "Dothraki":
        return dothraki_eval
    else:
        return None

def evaluate_dataset_with_delay(language, dataset):
    global df_global
    examples_text = build_example_string(language)
    items = list(dataset.items())
    total_examples = len(items)
    batches = [items[i:i + batch_size] for i in range(0, total_examples, batch_size)]
    results = []


    print(f"Starting evaluation for {language} with {total_examples} examples...")
    print(f"Processing {batch_size} examples per batch, waiting 60 seconds between batches.\n")

    initial_llm_call(language, examples_text)

    for i, batch in enumerate(batches, start=1):
        print(f"--- Batch {i}/{len(batches)} ---")

        for eng_phrase, expected in batch:
            prompt = f"""
    Using the previously provided examples,

    Translate:
    "{eng_phrase}"

    Output **only** the translated phrase, do not include any punctuation marks at the end of each phrase
    """
            try:
              response = chat.send_message(prompt)
            except:
              print("Waiting 60 seconds for quota limit reset")
              time.sleep(60)
              response = chat.send_message(prompt)

            predicted = response.text.strip()
            results.append({
                "input": eng_phrase,
                "expected": expected,
                "predicted": predicted
            })

        if i < len(batches):
            print("\n⏳ Waiting 60 seconds before the next batch...\n")
            time.sleep(60)

    df_global = pd.DataFrame(results)

    df_global.to_csv(f"{language.lower()}_eval_results.csv", index=False)
    print(f"\nResults saved as {language.lower()}_eval_results.csv")

    return df_global

def on_click(b):
    with output:
        output.clear_output()
        language = dropdown.value

        if language == "Select a language":
            print("Please select a valid language.")
            return

        dataset = load_eval_dataset(language)
        if dataset is None:
            print(f"No dataset found for {language}.")
            return

        df = evaluate_dataset_with_delay(language, dataset)

button.on_click(on_click)

display(dropdown, button, output)

Dropdown(description='Language:', options=('Select a language', "Na'vi", 'Klingon', 'Elvish', 'Dothraki'), val…

Button(description='Confirm', disabled=True, style=ButtonStyle())

Output()

This evaluation cell `Evaluating Translations using BERT Semantic Similarity Scores` provides a quantitative measurement of how well the model’s translated outputs align with the expected phrases from each fictional language. After the LLM has generated translations for all test sentences, this cell computes semantic similarity scores using a multilingual BERT model.

The process embeds both the expected translation and the model-predicted translation into a shared vector space and compares them using cosine similarity. Higher scores indicate that the two sentences carry similar meaning, even if their exact wording differs.

The next cell then provides the `Mean Accuracy` of the predicted outputs.

In [ ]:
#@title **Evaluating Translations using BERT Semantic Similarity Scores**
df_local = df_global.copy()

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

def embed_sentence(sentence):
    inputs = tokenizer(sentence, return_tensors='pt', truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

results = []

for _, row in tqdm(df_local.iterrows(), total=len(df_local)):
    reference = str(row["expected"])
    candidate = str(row["predicted"])
    input_text = str(row["input"])

    ref_embed = embed_sentence(reference)
    cand_embed = embed_sentence(candidate)
    cosine_sim = torch.nn.functional.cosine_similarity(ref_embed, cand_embed).item()

    results.append({
        "input": input_text,
        "expected": reference,
        "predicted": candidate,
        "bert_score": cosine_sim
    })

evaluation_df = pd.DataFrame(results)

pd.set_option('display.max_rows', None)
display(evaluation_df)

evaluation_df.to_csv("translation_eval_results.csv", index=False)

NameError: name 'df_global' is not defined

In [ ]:
#@title **Mean Accuracy of Predicted Translations**
print(f'Accuracy: {evaluation_df["bert_score"].mean()*100:.3f}%')

Accuracy: 71.022%


This `Try the Translation Yourself` cell is where you can try translating phrases yourself using the LLM and the in-context learning setup. It allows you to enter any English phrase and see how the model translates it into one of the fictional movie languages—Na’vi, Klingon, Elvish, or Dothraki—based solely on the examples provided in the prompt.

In [ ]:
#@title **Try the Translation Yourself!**
dropdown = widgets.Dropdown(
    options=["Select a language", "Na'vi", "Klingon", "Elvish", "Dothraki"],
    value="Select a language",
    description="Language:"
)

phrase = widgets.Text(
    placeholder="Enter a phrase",
    description="Phrase:"
)

button = widgets.Button(
    description="Confirm",
    disabled=True
)

output = widgets.Output()

def update_button(*args):
    button.disabled = not (dropdown.value != "Select a language" and phrase.value.strip())

dropdown.observe(update_button, 'value')
phrase.observe(update_button, 'value')

def build_example_string(language):
    if language == "Na'vi":
        data = navi_phrases
    elif language == "Klingon":
        data = klingon_phrases
    elif language == "Elvish":
        data = elvish_phrases
    elif language == "Dothraki":
        data = dothraki_phrases
    else:
        return ""

    return "\n".join(f"\"{eng}\" >> \"{trans}\"" for eng, trans in data.items())

def movie_translation(language, phrase):
    examples_text = build_example_string(language)

    prompt = f"""
    You are an expert translator specializing in a fictional movie language.

    You will be given:
    - A series of example translation pairs (English → fictional language {language})
    - A new English phrase to translate

    Your goals:
    - Infer the fictional language's rules from the examples
    - Translate the new English phrase with correct grammar and style
    - Maintain consistency with the patterns in the examples
    - Do NOT explain your reasoning or reveal the fictional language rules
    - Output **only** the translated phrase

    Examples:
    {examples_text}

    Translate:
    "{phrase}"
    """

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=[prompt],
    )
    return response.text

last_translation = {"text": None}

def on_click(b):
    with output:
        output.clear_output()
        language = dropdown.value
        user_phrase = phrase.value

        print(f'\nTranslating "{user_phrase}" to {language} ...')
        translated = movie_translation(language, user_phrase)
        print(f'\n"{user_phrase}" in {language} ➡️ {translated}')

        last_translation["text"] = translated

button.on_click(on_click)

display(dropdown, phrase, button, output)

Dropdown(description='Language:', options=('Select a language', "Na'vi", 'Klingon', 'Elvish', 'Dothraki'), val…

Text(value='', description='Phrase:', placeholder='Enter a phrase')

Button(description='Confirm', disabled=True, style=ButtonStyle())

Output()

## **Summary**

This demo illustrates how LLMs can perform in-context learning by recognizing and extending linguistic patterns from a limited set of example translations. By providing the model with English ↔ fictional-language phrase pairs and presenting them as few-shot demonstrations, we give the LLM an opportunity to infer vocabulary, syntax, and stylistic patterns without any explicit description of the underlying grammar. When prompted with new English phrases—ones not seen in the examples—the model is asked to translate them consistently with the learned patterns. Through this setup, we observe how the LLM captures the structure and style of the fictional language, relying on pattern induction rather than pre-programmed linguistic rules or memorization.

Beyond the specific languages used here—Na’vi, Klingon, Dothraki, and Elvish—the same methodology can be applied to other constructed languages or specialized jargon. By crafting appropriate few-shot demonstrations, LLMs can be guided to learn mappings for novel vocabulary, idiomatic expressions, or domain-specific phrases. These extensions demonstrate the flexibility of the in-context learning framework when applied to translation tasks, revealing how LLMs can generalize from limited examples to produce fluent, contextually appropriate translations.

## **Conclusion**

This demonstration highlights that LLMs can generalize structured linguistic behavior using only a handful of example translations—capturing the essence of in-context learning in the domain of fictional language translation. Even without access to dictionaries, grammar rules, or prior knowledge of the invented languages, the model can infer word mappings, sentence structure, and stylistic patterns, producing coherent translations for unseen phrases. While the LLM does not perform formal linguistic analysis, its ability to internalize and reproduce patterns shows strong potential for tasks requiring rapid adaptation to new languages, consistent style transfer, or low-resource translation scenarios. Translation of fictional languages thus serves as a compelling testbed for understanding both the strengths and limitations of treating LLMs as general pattern machines.